In [ ]:
!pip install dtreeviz

In [ ]:
import pandas as pd # data manipulation\n",
pd.options.mode.chained_assignment = None  # default='warn'\n",
import numpy as np #  mathematical support for large, multi-dimensional arrays and matrices\n",
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import re
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt

import collections
from matplotlib.patches import Arc
import math
import xgboost
import dtreeviz
from sklearn.model_selection import KFold


In [ ]:

## Import up sound alert dependencies
from IPython.display import Audio, display

def allDone():
  display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))
## Insert whatever audio file you want above

# allDone()


### Set Directory

In [ ]:
try:
    os.chdir('/kaggle/input/football-event-data')
except:
    pass

In [ ]:
try:
    os.chdir('/group/interns202010/jmakins/Data')
except:
    pass

In [ ]:
try:
    os.chdir('/Users/jordanmakins/Desktop/Data/Players')
except:
    pass

In [ ]:

## When USING KAGGLE

os.chdir('/kaggle/input/football-event-data')

#Teams = pd.read_json('teams.json')
#Competitions = pd.read_json('competitions.json')
Players = pd.read_json('players.json')
#Coaches = pd.read_json('coaches.json') # Managers

England = pd.read_json('matches_England.json')
France = pd.read_json('matches_France.json')
Italy = pd.read_json('matches_Italy.json')
Spain = pd.read_json('matches_Spain.json')
Germany = pd.read_json('matches_Germany.json')

England['Country'] = 'England'
France['Country'] = 'France'
Italy['Country'] = 'Italy'
Spain['Country'] = 'Spain'
Germany['Country'] = "Germany"

In [ ]:
try:
    os.chdir('/Users/jordanmakins/Desktop/Data')
except:
    pass


Teams = pd.read_json('teams.json')
#Competitions = pd.read_json('competitions.json')
Players = pd.read_json('players.json')
#Coaches = pd.read_json('coaches.json') # Managers

In [ ]:
try:
    os.chdir('/Users/jordanmakins/Desktop/Data/matches')
except:
    pass
os.listdir()

In [ ]:
try:
    os.chdir('/group/interns202010/jmakins/Data/matches')
except:
    pass

try:
    os.chdir('/Users/jordanmakins/Desktop/Data/matches')
except:
    pass

England = pd.read_json('matches_England.json')
France = pd.read_json('matches_France.json')
Italy = pd.read_json('matches_Italy.json')
Spain = pd.read_json('matches_Spain.json')
Germany = pd.read_json('matches_Germany.json')
#World_Cup = pd.read_json('matches_World_Cup.json')
#Euro_Champs = pd.read_json('matches_European_Championship.json')

In [ ]:
# Adding country labels to match observations
England['Country'] = 'England'
France['Country'] = 'France'
Italy['Country'] = 'Italy'
Spain['Country'] = 'Spain'
Germany['Country'] = "Germany"
#World_Cup['Country'] = 'World Cup'
#Euro_Champs['Country'] = 'European Champs'

In [ ]:
try:
    os.chdir('/group/interns202010/jmakins/Data/events')
except:
    pass
try:
    os.chdir('/Users/jordanmakins/Desktop/Data/events')
except:
    pass


In [ ]:
Events_France = pd.read_json('events_France.json')
Events_Spain = pd.read_json('events_Spain.json')
Events_Germany = pd.read_json('events_Germany.json')
#Events_EuroChamps = pd.read_json('events_European_Championship.json')
#Events_World_Cup = pd.read_json('events_World_Cup.json')
Events_Italy = pd.read_json('events_Italy.json')
Events_England = pd.read_json('events_England.json')

### Data Cleaning

In [ ]:
Matches = pd.concat([England, France, Italy, Spain, Germany], ignore_index=True)

In [ ]:
Matches.drop(['duration'], inplace=True, axis =1) # removing groupName variable from World Cups

In [ ]:
Players.drop(["passportArea"], inplace = True, axis = 1)

In [ ]:
Players = Players.rename(columns ={'wyId': 'playerId'}) # rename for convenience

In [ ]:
Teams = Teams.rename(columns = {'wyId': 'teamId'})
Players = Players.rename(columns = {'currentTeamId': 'teamId'})
Players = pd.merge(Players,Teams[['teamId', 'officialName']], on = 'teamId').rename(columns = {'officialName': 'clubName'})

In [ ]:
Players['Position'] = Players.role.apply(pd.Series)['code3'].values

In [ ]:
Players['birthCountry'] = Players.birthArea.apply(pd.Series)['name'].values

In [ ]:
Players.drop(["birthArea", 'role'], inplace = True, axis = 1)

### Manipulate Match Data

In [ ]:
teams = [list(x.keys()) for x in Matches.teamsData] # create two columns for identifying teams in match

In [ ]:
teams = [list(x.keys()) for x in Matches.teamsData] # create two columns for identifying teams in match
homeTeam, awayTeam = [],[]
for x in teams:
    homeTeam.append(x[0])
    awayTeam.append(x[1])

In [ ]:
# add columns for additional match statistics
Matches["homeManagerId"] = None
for name in ["Score", "ScoreHT", "ScoreET", "ScoreP"]:
    colname = "home" + name
    colname2 = "away" + name
    Matches[colname] = None
    Matches[colname2] = None

In [ ]:
# parse Home and Away teams as features from nest dictionary
teams = [list(x.keys()) for x in Matches.teamsData] # create two columns for identifying teams in match
homeTeam, awayTeam = [],[]
for game, team in enumerate(teams):
    if Matches.teamsData[game:game+1][game][team[0]]['side'] == 'home':
        homeTeam.append(team[0])
        awayTeam.append(team[1])
    else:
        awayTeam.append(team[0])
        homeTeam.append(team[1])

In [ ]:
def get_team_stats(df, team): # function to parse team match stats from dictionary embedded in column teamsData from the Matches dataframe
    
    Bench, Starters, Manager, Score, ScoreHT, ScoreET, ScoreP, teamId = [],[],[],[],[],[],[],[]
    
    for game in range(0, len(df)):

        Bench.append(list(pd.DataFrame.from_dict(df.teamsData[game][team[game]]['formation']['bench'])['playerId']))
        Starters.append(list(pd.DataFrame.from_dict(df.teamsData[game][team[game]]['formation']['lineup'])['playerId']))
        Manager.append(df.teamsData[game][team[game]]['coachId'])
        Score.append(df.teamsData[game][team[game]]['score'])
        ScoreHT.append(df.teamsData[game][team[game]]['scoreHT'])
        ScoreET.append(df.teamsData[game][team[game]]['scoreET'])
        ScoreP.append(df.teamsData[game][team[game]]['scoreP'])
        teamId.append(df.teamsData[game][team[game]]['teamId'])
    
    return Bench, Starters, Manager, Score, ScoreHT, ScoreET, ScoreP, teamId

homeBench, homeStarters, homeManager, homeScore, homeScoreHT, homeScoreET, homeScoreP, homeTeamId = get_team_stats(Matches, homeTeam)
awayBench, awayStarters, awayManager, awayScore, awayScoreHT, awayScoreET, awayScoreP, awayTeamId = get_team_stats(Matches, awayTeam)


In [ ]:
# Automating Bench Line Up Column Generation

homeColNames = []
for num in range(1,14): # add columns for home bench playerId
     homeColNames.append("homeBenchPlayer" + str(num))
        
awayColNames = []
for num in range(1,14): # add columns for away bench playerId
    awayColNames.append("awayBenchPlayer" + str(num))
    


In [ ]:
# Inspecting bench counts as during errror handling reveals some matches had bench sizes of between 8 and 13 for certain fixtures
import collections
collections.Counter(list(map(lambda x: len(x), homeBench)))

In [ ]:
# Adding Bench Line Ups to DataFrame

homeBench2 = pd.DataFrame(homeBench,
     columns=homeColNames)
awayBench2 = pd.DataFrame(awayBench,
     columns=awayColNames)


In [ ]:
# Automating Starting Line Up Column Generation

homeColNames =[]
for num in range(1,12): # add columns for starting playerId
     homeColNames.append("homePlayer" + str(num))
        
awayColNames = []
for num in range(1,12): # add columns for starting playerId
    awayColNames.append("awayPlayer" + str(num))


In [ ]:
# Joining starting line ups, benches to matches dataframe

homeStarters2 = pd.DataFrame(homeStarters,
     columns=homeColNames)
awayStarters2 = pd.DataFrame(awayStarters,
     columns=awayColNames)

Matches = Matches.join([homeStarters2, awayStarters2, homeBench2, awayBench2])


In [ ]:
# add individual columns of match data
Matches['homeManager'] = homeManager
Matches['homeScore'] = homeScore
Matches['homeScoreHT'] = homeScoreHT
Matches['homeScoreET'] = homeScoreET
Matches['homeScoreP'] = homeScoreP
Matches['homeTeamId'] = homeTeamId
Matches['awayManager'] = awayManager
Matches['awayScore'] = awayScore
Matches['awayScoreHT'] = awayScoreHT
Matches['awayScoreET'] = awayScoreET
Matches['awayScoreP'] = awayScoreP
Matches['awayTeamId'] = awayTeamId

In [ ]:
Matches.drop(["teamsData"], inplace = True, axis = 1)

In [ ]:
# Classify Match Result as W, L or D for teams
conditions = [
    (Matches['winner'] == 0),
    (Matches['winner'] == Matches['awayTeamId']),
    (Matches['winner'] == Matches['homeTeamId'])
    ]

# create a list of the values we want to assign for each condition
values = [0, -1, 1]

# create a new column and use np.select to assign values to it using our lists as arguments
Matches['Result'] = np.select(conditions, values)

In [ ]:
Matches["date"] = pd.to_datetime(Matches['dateutc']).dt.date # create a date column for Matches dataframe

In [ ]:
Matches["time"] = pd.to_datetime(Matches['dateutc']).dt.time # create a time column for Matches dataframe

In [ ]:
# Prelim Dimensionality Reduction
cols2Drop = ['status', 'roundId', 'gameweek', 'dateutc', 'label', 'referees', 'homeManagerId', 'seasonId']
Matches.drop(cols2Drop, inplace=True, axis =1)
Matches.drop(['winner', 'date', 'time'], inplace = True, axis =1)

In [ ]:
Matches.rename(columns={'wyId':'matchId'}, inplace=True)

In [ ]:
Matches = Matches.fillna(0) # fill bench7players with Ids = 0 in order to prevent program crashing

### Events Manipulation for Match/Player Stats and then combine with existing Match(test) dataframe from aboves

In [ ]:
# Decroos Github
tags = [
    (101, "goal"),
    (102, "own_goal"),
    (301, "assist"),
    (302, "key_pass"),
    (1901, "counter_attack"),
    (401, "left_foot"),
    (402, "right_foot"),
    (403, "head/body"),
    (1101, "direct"),
    (1102, "indirect"),
    (2001, "dangerous_ball_lost"),
    (2101, "blocked"),
    (801, "high"),
    (802, "low"),
    (1401, "interception"),
    (1501, "clearance"),
    (201, "opportunity"),
    (1301, "feint"),
    (1302, "missed_ball"),
    (501, "free_space_right"),
    (502, "free_space_left"),
    (503, "take_on_left"),
    (504, "take_on_right"),
    (1601, "sliding_tackle"),
    (601, "anticipated"),
    (602, "anticipation"),
    (1701, "red_card"),
    (1702, "yellow_card"),
    (1703, "second_yellow_card"),
    (1201, "position_goal_low_center"),
    (1202, "position_goal_low_right"),
    (1203, "position_goal_mid_center"),
    (1204, "position_goal_mid_left"),
    (1205, "position_goal_low_left"),
    (1206, "position_goal_mid_right"),
    (1207, "position_goal_high_center"),
    (1208, "position_goal_high_left"),
    (1209, "position_goal_high_right"),
    (1210, "position_out_low_right"),
    (1211, "position_out_mid_left"),
    (1212, "position_out_low_left"),
    (1213, "position_out_mid_right"),
    (1214, "position_out_high_center"),
    (1215, "position_out_high_left"),
    (1216, "position_out_high_right"),
    (1217, "position_post_low_right"),
    (1218, "position_post_mid_left"),
    (1219, "position_post_low_left"),
    (1220, "position_post_mid_right"),
    (1221, "position_post_high_center"),
    (1222, "position_post_high_left"),
    (1223, "position_post_high_right"),
    (901, "through"),
    (1001, "fairplay"),
    (701, "lost"),
    (702, "neutral"),
    (703, "won"),
    (1801, "accurate"),
    (1802, "not_accurate"),
]

In [ ]:
tags = dict(tags)

In [ ]:
# ## I have kept this cell encase we need to impute integers for ML algorithms as oppose to strings
# event_tag_ids = []
# for ids in  list(trial[0:10]['tags']):
#     event_tag_ids.append(list(map(lambda y: y['id'], ids)))
# trial['event_tag_ids'] = event_tag_ids

In [ ]:
# Store event type tags ids in new column
def event_tags(event_df):
    event_tags = []
    for ids in list(event_df['tags']):
        event_tags.append(list(map(lambda y: tags[y['id']], ids)))

    event_df['event_tags'] = event_tags
    return event_df

In [ ]:
def RunningTime(event_df):
    # Solution to convert second half time into 
    secs_to_add = list(event_df[event_df['matchPeriod']=="1H"].groupby('matchId').tail(1)['eventSec'])
    match_ids = list(event_df["matchId"].unique())

    for idx in range(0, len(match_ids)):
        event_df['eventSec'] = list(np.where(
           (event_df['matchId'] == match_ids[idx]) & (event_df['matchPeriod'] == "2H") , event_df['eventSec'] + secs_to_add[idx], event_df['eventSec']
           ))
    return event_df


In [ ]:
# Clean the result of the action
def Result(event_df):
    result = []
    for tag in event_df["event_tags"]:
        if "accurate" in tag:
            result.append("Accurate")
        elif "not_accurate" in tag:
            result.append("Inaccurate")
        else:
            result.append("")
    event_df["Result"] = result 
    return event_df

In [ ]:
# Separating description from success/failure of an action (event)
def event_description(event_df):
    subEventDescr = []
    for tag in event_df["event_tags"]:
        descr = " ".join(tag[:-1])
        if descr != "":
            subEventDescr.append(descr)
        else:
            subEventDescr.append("generic play")
    event_df["subEventDescription"] = subEventDescr
    event_df = event_df[(event_df["event_tags"].str.len() != 0) & (event_df.subEventName != 'Ball out of the field')  & (event_df.subEventName != 'Goal kick')  ]
    event_df = event_df[event_df["subEventName"] != "Throw in"] # remove throw-ins as a relevant feature among successful teams for simpler analysis, unless its Rory Delap!
    return event_df

In [ ]:
# # Separating description from success/failure of an action (event)
# def event_description(event_df):
#     subEventDescr = []
#     for tag in event_df["event_tags"]:
#         descr = " ".join(tag[:-1])
#         if descr != "":
#             subEventDescr.append(descr)
#         else:
#             subEventDescr.append("generic play")
#     event_df["subEventDescription"] = subEventDescr
#     event_df = event_df[(event_df["event_tags"].str.len() != 0) & (event_df.subEventName != 'Ball out of the field')  & (event_df.subEventName != 'Goal kick') & (event_df.subEventName != 'Touch')   ]
#     event_df = event_df[event_df["subEventName"] != "Throw in"] # remove throw-ins as a relevant feature among successful teams for simpler analysis, unless its Rory Delap!
#     return event_df

In [ ]:

def Locations(event_df):

    xStart,xEnd,yStart,yEnd = [], [], [], []
    for pos in event_df["positions"]:
        if len(pos) == 2:
            xStart.append(pos[0]['x'])
            xEnd.append(pos[1]['x'])
            yStart.append(pos[0]['y'])
            yEnd.append(pos[1]['y'])
        else:
            xStart.append(pos[0]['x'])
            xEnd.append(pos[0]['x'])
            yStart.append(pos[0]['y'])
            yEnd.append(pos[0]['y'])


    event_df['xStart'], event_df['xEnd'], event_df['yStart'], event_df['yEnd'] = xStart, xEnd, yStart, yEnd
    return event_df

In [ ]:
# function to transform an event data frame
def event_df_transform(event_df):
    event_df = event_tags(event_df)
    event_df = Result(event_df)
    event_df = event_description(event_df)
    event_df = Locations(event_df)
    event_df.drop(['positions', "event_tags", "tags", "eventName"], inplace = True, axis = 1) #'id'
    event_df = event_df[event_df.subEventName != ""]
    event_df = event_df[event_df['playerId']!= 0]
    event_df = RunningTime(event_df)
    event_df['attackMetres'] = event_df['xEnd'] - event_df['xStart']
    return event_df

# stopped remove eventId
    

In [ ]:
# Transform event_dfs

Events_France = event_df_transform(Events_France)
Events_Spain = event_df_transform(Events_Spain)
Events_Germany = event_df_transform(Events_Germany)
Events_Italy = event_df_transform(Events_Italy)
Events_England = event_df_transform(Events_England)


In [ ]:
def zones(event_df, start_or_end):

    conditions = [
            (((event_df['x'+ start_or_end] < 16.5)& (event_df['x'+ start_or_end] >= 0) & (event_df['y'+ start_or_end] >= 0)& (event_df['y'+ start_or_end] <= 25))),
            (((event_df['x'+ start_or_end] < 16.5)& (event_df['x'+ start_or_end] >= 0) & (event_df['y'+ start_or_end] > 25)& (event_df['y'+ start_or_end] <= 36))),
            (((event_df['x'+ start_or_end] < 16.5)& (event_df['x'+ start_or_end] >= 0) & (event_df['y'+ start_or_end] > 36)& (event_df['y'+ start_or_end] <= 64))),
            (((event_df['x'+ start_or_end] < 16.5)& (event_df['x'+ start_or_end] >= 0) & (event_df['y'+ start_or_end] > 64)& (event_df['y'+ start_or_end] <= 75))),
            (((event_df['x'+ start_or_end] < 16.5)& (event_df['x'+ start_or_end] >= 0) & (event_df['y'+ start_or_end] > 75)& (event_df['y'+ start_or_end] <= 100))),
        
            (((event_df['x'+ start_or_end] <= 33)& (event_df['x'+ start_or_end] >= 0) & (event_df['y'+ start_or_end] >= 0)& (event_df['y'+ start_or_end] <=  25))),
            (((event_df['x'+ start_or_end] <= 33)& (event_df['x'+ start_or_end] >= 0) & (event_df['y'+ start_or_end] > 25)& (event_df['y'+ start_or_end] <=  50))),
            (((event_df['x'+ start_or_end] <= 33)& (event_df['x'+ start_or_end] >= 0) & (event_df['y'+ start_or_end] > 50)& (event_df['y'+ start_or_end] <=  75))),
            (((event_df['x'+ start_or_end] <= 33)& (event_df['x'+ start_or_end] >= 0) & (event_df['y'+ start_or_end] > 75)& (event_df['y'+ start_or_end] <=  100))),
            (((event_df['x'+ start_or_end] <= 50)& (event_df['x'+ start_or_end] > 33) & (event_df['y'+ start_or_end] >= 0))& (event_df['y'+ start_or_end] <=  25)),
            (((event_df['x'+ start_or_end] <= 50)& (event_df['x'+ start_or_end] > 33) & (event_df['y'+ start_or_end] > 25)& (event_df['y'+ start_or_end] <=  50))),
            (((event_df['x'+ start_or_end] <= 50)& (event_df['x'+ start_or_end] > 33) & (event_df['y'+ start_or_end] > 50)& (event_df['y'+ start_or_end] <=  75))),
            (((event_df['x'+ start_or_end] <= 50)& (event_df['x'+ start_or_end] > 33) & (event_df['y'+ start_or_end] > 75)& (event_df['y'+ start_or_end] <=  100))),
            (((event_df['x'+ start_or_end] <= 67)& (event_df['x'+ start_or_end] > 50) & (event_df['y'+ start_or_end] >= 0)& (event_df['y'+ start_or_end] <= 25))),
            (((event_df['x'+ start_or_end] <= 67)& (event_df['x'+ start_or_end] > 50) & (event_df['y'+ start_or_end] > 25)& (event_df['y'+ start_or_end] <=  50))),
            (((event_df['x'+ start_or_end] <= 67)& (event_df['x'+ start_or_end] > 50) & (event_df['y'+ start_or_end] > 50)& (event_df['y'+ start_or_end] <= 75))),
            (((event_df['x'+ start_or_end] <= 67)& (event_df['x'+ start_or_end] > 50) & (event_df['y'+ start_or_end] > 75)& (event_df['y'+ start_or_end] <= 100))),
            (((event_df['x'+ start_or_end] < 83.5)& (event_df['x'+ start_or_end] > 67) & (event_df['y'+ start_or_end] >= 0)& (event_df['y'+ start_or_end] <= 25))),
            (((event_df['x'+ start_or_end] < 83.5)& (event_df['x'+ start_or_end] > 67) & (event_df['y'+ start_or_end] > 25)& (event_df['y'+ start_or_end] <= 50))),
            (((event_df['x'+ start_or_end] < 83.5)& (event_df['x'+ start_or_end] > 67) & (event_df['y'+ start_or_end] > 50)& (event_df['y'+ start_or_end] <= 75))),
            (((event_df['x'+ start_or_end] < 83.5)& (event_df['x'+ start_or_end] > 67) & (event_df['y'+ start_or_end] > 75)& (event_df['y'+ start_or_end] <= 100))),
        
            (((event_df['x'+ start_or_end] <= 100) & (event_df['x'+ start_or_end] > 83.5) & (event_df['y'+ start_or_end] >= 0) & (event_df['y'+ start_or_end] <= 25))),
            (((event_df['x'+ start_or_end] <= 100) & (event_df['x'+ start_or_end] > 83.5) & (event_df['y'+ start_or_end] > 25) & (event_df['y'+ start_or_end] <=  36))),
            (((event_df['x'+ start_or_end] <= 100) & (event_df['x'+ start_or_end] > 83.5) & (event_df['y'+ start_or_end] > 36) & (event_df['y'+ start_or_end] <= 64))),
            (((event_df['x'+ start_or_end] <= 100) & (event_df['x'+ start_or_end] > 83.5) & (event_df['y'+ start_or_end] > 64) & (event_df['y'+ start_or_end] <= 75))),
            (((event_df['x'+ start_or_end] <= 100) & (event_df['x'+ start_or_end] > 83.5) & (event_df['y'+ start_or_end] > 75) & (event_df['y'+ start_or_end] <= 100)))
            ]

    # create a list of the values we want to assign for each condition
    values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]

    # create a new column and us np.select to assign values to it using our lists as arguments
    event_df['Zone'+ start_or_end] = np.select(conditions, values)
    
    return event_df



        

In [ ]:
for event_df in [Events_England, Events_Italy, Events_Spain, Events_France, Events_Germany]:
    event_df = zones(event_df, "Start")
    event_df = zones(event_df, "End")

In [ ]:
### CHange German row with 101
Events_Germany.loc[Events_Germany.yEnd==101, ['yEnd']] = 99

In [ ]:
### remove events where playerId equals zero
## There are two instances where goals are scored so we will handle these first and assign to correct players as these are important events
# we can attempt to handle these into the real player sequences if we have time at the end of the project


In [ ]:
# function to determine average player coordinates on pitch across all possessions in a match
def playerPosition(event_df):
    df = event_df.groupby(['matchId','playerId']).agg({'xStart': ['mean'], 'yStart': ['mean']}).reset_index()
    df.columns = ["matchId", "playerId", "xStart", "yStart"]
    return df

In [ ]:

# find positions for all players for big 5 Euro leagues
a = playerPosition(Events_England)
b = playerPosition(Events_France)
c = playerPosition(Events_Italy)
d = playerPosition(Events_Spain)
e = playerPosition(Events_Germany)

position_df = pd.concat([a,b,c,d,e])


In [ ]:

# This dataframe is used relationally to fill correct team as events represent 2018 team, while player
a = Events_England[['playerId', "teamId", 'matchId']].groupby(['playerId', "teamId", 'matchId']).count().reset_index()
b = Events_France[['playerId', "teamId", 'matchId']].groupby(['playerId', "teamId", 'matchId']).count().reset_index()
c = Events_Italy[['playerId', "teamId", 'matchId']].groupby(['playerId', "teamId", 'matchId']).count().reset_index()
d = Events_Spain[['playerId', "teamId", 'matchId']].groupby(['playerId', "teamId", 'matchId']).count().reset_index()
e = Events_Germany[['playerId', "teamId", 'matchId']].groupby(['playerId', "teamId", 'matchId']).count().reset_index()
p_refs = pd.concat([a,b,c,d,e])


In [ ]:

# Create PA dataframe
Player_Aggs = Players[['shortName','Position', 'playerId','weight','height',  'foot' ]]

# add ave field locations to players
Player_Aggs = pd.merge(Player_Aggs, position_df, how = 'right', on= 'playerId').sort_values('matchId')
Player_Aggs = pd.merge(Player_Aggs, p_refs, on = ['matchId', 'playerId'])
Player_Aggs = pd.merge(Player_Aggs, Teams[['teamId', 'name']], on = 'teamId')
Player_Aggs = pd.merge(Player_Aggs, Matches[['matchId', 'homeTeamId', 'awayTeamId']], on = 'matchId')
Player_Aggs.loc[Player_Aggs.teamId == Player_Aggs.homeTeamId, 'homeAway'] = "home"
Player_Aggs.loc[Player_Aggs.teamId == Player_Aggs.awayTeamId, 'homeAway'] = "away"


In [ ]:
# # # # # #

# var = "Shot"
# varName = "ShotsOnTarget"

# # this exists
# df = Events_England.loc[(Events_England.Result == "Accurate" ) & (Events_England.subEventName== var )].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='ShotsOnTarget')#.groupby(["matchId",'playerId']).agg(list).reset_index()
# # [['playerId', 'matchId', "ZoneStart"]]
# # # # # # # 

def transform_zonal(df, varName):

    df = df.pivot_table(varName, ['playerId', 'matchId'], 'ZoneStart').fillna(0).reset_index()

    column_indices = []
    new_names = []
    check_cols = list(range(1,27))
    old_columns = list(df.columns[2:])

    for num, col in enumerate(old_columns):
        column_indices.append(num+2)
        new_names.append(varName+ "_ZoneS_" + str(col))

    # columns to be added 
    change_cols = np.setdiff1d(check_cols, old_columns) 

    old_names = df.columns[column_indices]
    df.rename(columns=dict(zip(old_names, new_names)), inplace=True)

    for col in change_cols:
        df[varName + "_ZoneS_" + str(col)] = 0

    return df




In [ ]:
### unique entry dataframe for player gameTime at any point in a match by summarizing streams

def playerGameTime(event_df):
    df = event_df.groupby(['matchId','playerId'])['eventSec'].agg(['max', 'min']).reset_index()
    df['gameTime (min)'] = round((df['max'] - df['min'])/60) # game time to the nearest minute
    df.drop(['max', 'min'], inplace=True, axis=1)
    return df

# derive gameTime per player and store in summary table
a = playerGameTime(Events_England)
b = playerGameTime(Events_France)
c = playerGameTime(Events_Italy)
d = playerGameTime(Events_Spain)
e = playerGameTime(Events_Germany)

playingTime = pd.concat([a,b,c,d,e]).fillna(0)


In [ ]:
def find_goals(event_df):

    # Create a column to recognize when a goal is scored
    conditions = [
        (((event_df['subEventDescription'].str.match('goal ')) & (event_df['Result'] == "Accurate"))),
        (((event_df['subEventDescription'].str.match('goal ')) & (event_df['Result'] == "Inaccurate") & (event_df['subEventName'] == "Shot"))),
        (((event_df['subEventDescription'].str.match('goal ')) & (event_df['Result'] == "Inaccurate") & (event_df['subEventName'] != "Shot")))
        ]

    # create a list of the values we want to assign for each condition
    values = [1, 1, -1]

    # create a new column and use np.select to assign values to it using our lists as arguments
    event_df['Goal_Value'] = np.select(conditions, values, default = 0)
    #[['playerId', 'matchId', "ZoneStart"]].groupby(['playerId', 'matchId', "ZoneStart"])
    return event_df.groupby(['playerId', 'matchId', "ZoneStart"]).sum().reset_index()#[['playerId', 'matchId', 'Goal_Value']]

In [ ]:
for event in [Events_England, Events_Italy, Events_France, Events_Germany, Events_Spain]:
    event = find_goals(event)


In [ ]:
Player_Aggs = pd.merge(Player_Aggs, playingTime, how = 'left', on = ['playerId', 'matchId'])

In [ ]:
# convenience function for event_df parsing
def parseEvent_df(func, new_feature):
    
    aa = func(Events_England)
    aa = transform_zonal(aa, new_feature)
#     aa = aa.rename_axis(None, axis=1)
    bb = func(Events_France)
    bb = transform_zonal(bb, new_feature)
#     bb = bb.rename_axis(None, axis=1)
    cc = func(Events_Italy)
    cc = transform_zonal(cc, new_feature)
#     cc = cc.rename_axis(None, axis=1)
    dd = func(Events_Spain)
    dd = transform_zonal(dd, new_feature)
#     dd = dd.rename_axis(None, axis=1)
    ee = func(Events_Germany)
    ee = transform_zonal(ee, new_feature)
#     ee = ee.rename_axis(None, axis=1)
    
    return pd.concat([aa,bb,cc,dd,ee])

In [ ]:
# function for adding new feature column to Player_Aggs df
def alter_Player_Aggs(new_feature_function, new_feature, PA_df):
    
    if "Acc" in new_feature:
        df = parseEvent_df(new_feature_function, new_feature)
        #df = transform_zonal(df, new_feature)
        PA_df = pd.merge(PA_df, df, how= 'left', on =['playerId', 'matchId'])
#         new_cols = [col for col in df if col.startswith(new_feature)]
#         PA_df[new_cols] = PA_df[new_cols].fillna(0)
        
    elif 'acc' in new_feature:
        df = parseEvent_df(new_feature_function, new_feature)
        #df = transform_zonal(df, new_feature)
        PA_df = pd.merge(PA_df, df, how= 'left', on =['playerId', 'matchId'])
#         new_cols = [col for col in df if col.startswith(new_feature)]
#         PA_df[new_cols] = PA_df[new_cols].fillna(0)
        
    else:
        df = parseEvent_df(new_feature_function, new_feature)
        #df = transform_zonal(df, new_feature)
        PA_df = pd.merge(PA_df, df, how= 'left', on =['playerId', 'matchId'])
#         new_cols = [col for col in df if col.startswith(new_feature)]
#         PA_df[new_cols] = PA_df[new_cols].fillna(0)
        
#         PA_df[new_cols] = (PA_df[new_cols] / PA_df['gameTime (min)']) * 90 # get standardized stat index for 90mins
    PA_df.iloc[:,15:] = PA_df.iloc[:,15:].fillna(0)
    return PA_df


In [ ]:
# import datetime
# while datetime.datetime.now().hour < 17:
#     x = 1+1

In [ ]:
# calc angle to goal from any location in radians

def angleToGoal(event_df):
    
    event_df['C'] = 10 # length of goal
    event_df['A'] = ((100-event_df.xStart)**2+abs(45-event_df.yStart)**2)**.5
    event_df['B'] = ((100-event_df.xStart)**2+abs(55-event_df.yStart)**2)**.5
    event_df['angle_to_goal'] = (event_df['A'] * event_df['A'] + event_df['B'] * event_df['B'] - event_df['C'] * event_df['C'])/(2 * event_df['A'] * event_df['B'])
    event_df['angle_to_goal'] = event_df['angle_to_goal'].apply(lambda row: round(math.radians(math.degrees(math.acos(row))),4))
    # event_df['angle_to_goal'] = event_df['angle_to_goal'].apply(lambda row: round(math.degrees(row),2))
    event_df.drop(['A', "B", "C"], inplace = True, axis=1)
    return event_df
        
Events_England, Events_France = angleToGoal(Events_England), angleToGoal(Events_France)
Events_Germany, Events_Spain = angleToGoal(Events_Germany), angleToGoal(Events_Spain)
Events_Italy = angleToGoal(Events_Italy)


### Function to prepare dataframe for fdnn

In [ ]:
# result is event_df

def player_goals(result):
    
    conditions = [
            (((result['subEventDescription'].str.match('goal ')) & (result['Result'] == "Accurate"))),
            (((result['subEventDescription'].str.match('goal ')) & (result['Result'] == "Inaccurate") & (result['subEventName'] == "Shot"))),
            (((result['subEventDescription'].str.match('goal ')) & (result['Result'] == "Inaccurate") & (result['subEventName'] != "Shot")))
            ]

    # create a list of the values we want to assign for each condition
    values = [1, 1, -1]

    # create a new column and us np.select to assign values to it using our lists as arguments
    result['Goal_Value'] = np.select(conditions, values, default = 0)
    
    return result

    

In [ ]:
def corner_assists(event_df):
    return event_df[(event_df.subEventName=="Corner") & (event_df.subEventDescription.str.match('assist'))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='CornerAssists')

def corner_opportunity(event_df):
    return event_df[(event_df.subEventName=="Corner") & (event_df.subEventDescription.str.match('key_pass'))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='CornerOpportunitiesCreated')

def corner_success(event_df):
    return event_df[(event_df.subEventName=="Corner") & (event_df.Result=="Accurate")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='SuccessfulCorners')

def corner_fail(event_df):
    return event_df[(event_df.subEventName=="Corner") & (event_df.Result=="Inaccurate")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='FailedCorners')

def corner_assists(event_df):
    return event_df[(event_df.subEventName=="Corner") & (event_df.subEventDescription.str.match('assist'))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='CornerAssists')


In [ ]:
### Attack Metres stat (how many forward metres achieved through events)
def AccBackwardMetres(event_df):

    df = event_df.loc[(event_df.Result == "Accurate" ) & (event_df.attackMetres < 0 )].groupby(['playerId', 'matchId', "ZoneStart"])['attackMetres'].sum().reset_index().rename(columns={'attackMetres': "AccBackMetres"})
    df["AccBackMetres"] = df["AccBackMetres"].abs()
    return df

def InaccBackwardMetres(event_df):

    df = event_df.loc[(event_df.Result == "Inaccurate") & (event_df.attackMetres < 0 )].groupby(['playerId', 'matchId', "ZoneStart"])['attackMetres'].sum().reset_index().rename(columns={'attackMetres': "InaccBackMetres"})
    df["InaccBackMetres"] = df["InaccBackMetres"].abs()
    return df


### Attack Metres stat (how many forward metres achieved through events)
def AccForwardMetres(event_df):

    df = event_df.loc[(event_df.Result == "Accurate") & (event_df.attackMetres > 0 )].groupby(['playerId', 'matchId', "ZoneStart"])['attackMetres'].sum().reset_index().rename(columns={'attackMetres': "AccForwardMetres"})
    return df

def InaccForwardMetres(event_df):

    df = event_df.loc[(event_df.Result == "Inaccurate") & (event_df.attackMetres > 0 ) ].groupby(['playerId', 'matchId', "ZoneStart"])['attackMetres'].sum().reset_index().rename(columns={'attackMetres': "InaccForwardMetres"})
    return df


In [ ]:

# adding in total meterage of accurate and inaccurate movements by players in the vertical plane towards the goal
# AccBackward = parseEvent_df(AccBackwardMetres)
# InaccBackward = parseEvent_df(InaccBackwardMetres)
# AccForward = parseEvent_df(AccForwardMetres)
# InaccForward= parseEvent_df(InaccForwardMetres)

# Player_Aggs = pd.merge(Player_Aggs, AccBackward, how= 'left', on =['playerId', 'matchId'])
# Player_Aggs = pd.merge(Player_Aggs, InaccBackward, how = 'left', on =['playerId', 'matchId'])
# Player_Aggs = pd.merge(Player_Aggs, AccForward, how= 'left', on =['playerId', 'matchId'])
# Player_Aggs = pd.merge(Player_Aggs, InaccForward, how = 'left', on =['playerId', 'matchId'])
# Player_Aggs.AccForwardMetres = Player_Aggs.AccForwardMetres.fillna(0)
# Player_Aggs.InaccForwardMetres = Player_Aggs.InaccForwardMetres.fillna(0)-
# Player_Aggs.AccBackMetres = Player_Aggs.AccBackMetres.fillna(0)
# Player_Aggs.InaccBackMetres = Player_Aggs.InaccBackMetres.fillna(0)


##### Explore Feature

In [ ]:
Events_England.subEventName.unique()

#### Miscellanous 

In [ ]:
def SimulationFouls(event_df):
    df = event_df[(event_df.subEventName=="Simulation")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='SimulationFouls')
    return df

def Fouls(event_df):
    df= event_df[(event_df.subEventName.str.contains('foul'))| (event_df.subEventName.str.contains('Foul'))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='FoulsCommited')

    return df

def Clearances(event_df):
    df =  event_df[(event_df.subEventName=="Clearance")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='Clearances')

    return df
    
def AccLaunchMetres(event_df):
    df =  event_df.loc[(event_df.Result == "Accurate") &(event_df.subEventName=="Launch")].groupby(['playerId', 'matchId', "ZoneStart"])['attackMetres'].sum().reset_index().rename(columns={'attackMetres': "AccLaunchMetres"})

    return df
    
def InaccLaunchMetres(event_df):
    df = event_df.loc[(event_df.Result == "Inaccurate") &(event_df.subEventName=="Launch")].groupby(['playerId', 'matchId', "ZoneStart"])['attackMetres'].sum().reset_index().rename(columns={'attackMetres': "InaccLaunchMetres"})

    return df

def FreeKickCrossKey(event_df):
    df = event_df[(event_df.subEventName == "Free kick cross")&(event_df.subEventDescription.str.contains("key"))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='FKKeyCross')

    return df
        
def FreeKickCrossAssists(event_df):
    df = event_df[(event_df.subEventName == "Free kick cross")&(event_df.subEventDescription.str.contains("assist"))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='FKCrossAssists')
    return df
    
def FreeKickCrossAccuracy(event_df):
    df1= event_df[(event_df.subEventName == "Free kick cross")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='FreeKickCrosses')
    df2 = event_df[(event_df.subEventName == "Free kick cross")& (event_df.Result=="Accurate")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='AccFreeKickCrosses')

    return df2


In [ ]:
#penaltiesConversion Statistic

def PenaltiesConverted(event_df):
    df1= event_df[(event_df.subEventName == "Penalty")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='PenaltiesAttempted')

    df2 = event_df[(event_df.subEventName == "Penalty")& (event_df.Result=="Accurate")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='PenaltiesScored')

    df1 = pd.merge(df1,df2, how='left', on = ['matchId', 'playerId'])
    df1['PenaltiesScored'] = df1['PenaltiesScored'].fillna(0)
    df1['penaltiesConversion'] = df1['PenaltiesScored'] / df1['PenaltiesAttempted']
    df1.drop(['PenaltiesScored', 'PenaltiesAttempted' ], inplace=True, axis =1)


    return df1


In [ ]:
# 'Ground loose ball duel', 'Ground defending duel'
def dangerousOpponentHalfRecoveries(event_df):
    df = event_df[((event_df.subEventDescription.str.contains("won")) | (event_df.subEventDescription.str.contains("interception") & (event_df.Result=="Accurate"))) & (event_df.subEventName!="Ground attacking duel") & (event_df.xEnd > 60)].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='dangerousOpponentHalfRecoveries')
    return df
    
def dangerousOwnHalfBallLost(event_df):
    df = event_df[(event_df.Result=="Inaccurate") & (event_df.xEnd<40) & ((event_df.subEventName.str.contains("pass")) | (event_df.subEventName.str.contains("Acceleration")) | (event_df.subEventName.str.contains("Clearance")) | ((event_df.subEventName.str.contains("duel")) &(event_df.subEventName!='Ground defending duel')) | (event_df.subEventName.str.contains("Launch")))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='dangerousOwnHalfBallLost')
    return df
    
    
def dangerousDefDuelsLost(event_df):
    df = event_df[(event_df.Result=="Inaccurate") & (event_df.xEnd<40) & (event_df.subEventName=="Ground defending duel")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='dangerousDefDuelsLost')
    return df


In [ ]:
### Step one proportion of low left shots saved

possies = ['low_left', 'mid_left', 'high_left', 'low_center', 'mid_center', 'high_center', 'low_right', 'mid_right', 'high_right']

def GoalKeepingZoneEfficiency(event_df, pos):
    df1= event_df[((event_df.subEventName=="Save attempt") | (event_df.subEventName=="Reflexes")) &(event_df.subEventDescription.str.contains(pos))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='total_'+pos)

    df2 = event_df[((event_df.subEventName=="Save attempt") | (event_df.subEventName=="Reflexes")) &(event_df.subEventDescription.str.contains(pos)) & (event_df.Result=="Accurate")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name=pos+'_saves')

    df1 = pd.merge(df1,df2, how='left', on = ['matchId', 'playerId'])
    df1[pos+'_saves'] = df1[pos+'_saves'].fillna(0)
    df1[pos+'_save_efficiency'] = df1[pos+'_saves'] / df1['total_'+pos]
    df1.drop([pos+'_saves', 'total_'+pos], inplace=True, axis =1)
#     df1 = transform_zonal(df1, desiredName)
    return df1


# for pos in possies:
#     a = GoalKeepingZoneEfficiency(Events_England, pos)
#     b = GoalKeepingZoneEfficiency(Events_France, pos)
#     c = GoalKeepingZoneEfficiency(Events_Italy, pos)
#     d = GoalKeepingZoneEfficiency(Events_Spain, pos)
#     e = GoalKeepingZoneEfficiency(Events_Germany, pos)
#     df = pd.concat([a,b,c,d,e])
#     Player_Aggs = pd.merge(Player_Aggs, df, how= 'left', on = ['playerId', 'matchId'])
#     Player_Aggs[ pos + '_save_efficiency'] = Player_Aggs[pos + '_save_efficiency'].fillna(0)
    

In [ ]:
# Goals allowed
def goalsAllowed(event_df):
    df = event_df[((event_df.subEventName == "Reflexes")|(Events_England.subEventName == "Save attempt")) & (event_df.Result == "Inaccurate") ].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='goalsAllowed')
#     df = transform_zonal(df, desiredName)
    return df

# Successful save attempt
def Saves(event_df):
    df= event_df[((event_df.subEventName == "Save attempt")|(Events_England.subEventName == "Save attempt"))&(event_df.Result == "Accurate") ].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='GkSaves')
#     df = transform_zonal(df, desiredName)
    return df

# Accurate Hand Pass
def SuccessHandPass(event_df):
    df= event_df[(event_df.subEventName == "Hand pass")&(event_df.Result == "Accurate") ].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='AccurateHandPass')
#     df = transform_zonal(df, desiredName)
    return df
# Inaccurate Hand Pass
def FailedHandPass(event_df):
    df = event_df[(event_df.subEventName == "Hand pass")&(event_df.Result == "Inaccurate") ].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='InaccurateHandPass')
#     df = transform_zonal(df, desiredName)
    return df

# GoalKeeper Leaving Line 
def LeavingLine(event_df):
    df = event_df[(event_df.subEventName=="Goalkeeper leaving line")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='GKLeavingLineInstance')
#     df = transform_zonal(df, desiredName)
    return df


#### Crosses, Dribbles, Accelerations

In [ ]:
# Number of succesful accelerations
def SuccessAccelerations(event_df):
    df = event_df[(event_df.subEventName == 'Acceleration') & (event_df.Result == 'Accurate')].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='SuccessfulAccels')
#     df = transform_zonal(df, desiredName)
    return df

# Number of failed accelerations
def FailedAccelerations(event_df):
    df = event_df[(event_df.subEventName == 'Acceleration') & (event_df.Result == 'Inaccurate')].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='FailedAccels')
#     df = transform_zonal(df, desiredName)
    return df

# Total acceleration metres
def AccelDistance(event_df):
    df = event_df.loc[(event_df.Result == "Accurate") & (event_df.subEventName=="Acceleration") ].groupby(['playerId', 'matchId', "ZoneStart"])['attackMetres'].sum().reset_index().rename(columns={'attackMetres': "CumAccelerationDist"})
#     df = transform_zonal(df, desiredName)
    return df

# Accelerations into final third
def AccelsIntoFinalThird(event_df):
    df= event_df.loc[(event_df.Result == "Accurate") & (event_df.subEventName=="Acceleration") & (event_df.xStart <= 66)  & (event_df.xEnd > 66)].groupby(['playerId', 'matchId', "ZoneStart"])['attackMetres'].sum().reset_index().rename(columns={'attackMetres': "AccelsDistIntoFinal3rd"})
#     df = transform_zonal(df, desiredName)
    return df

# Acceleration within final third
def AccelsWithinFinalThird(event_df):
    df = event_df.loc[(event_df.Result == "Accurate") & (event_df.subEventName=="Acceleration") & (event_df.xStart > 66) ].groupby(['playerId', 'matchId', "ZoneStart"])['attackMetres'].sum().reset_index().rename(columns={'attackMetres': "AccelsDistWithinFinal3rd"})
#     df = transform_zonal(df, desiredName)
    return df


##### Crossing

In [ ]:
# cross assists
def crossAssists(event_df):
    df = event_df[(event_df.subEventName == 'Cross') & (event_df.subEventDescription.str.contains('assist'))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='CrossAssists')
    df = transform_zonal(df, desiredName)
    return df

# right foot crosses
def RightFootCross(event_df):
    df = event_df[(event_df.subEventName == 'Cross') & (event_df.subEventDescription.str.contains('right_foot'))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='RightFootCross')
#     df = transform_zonal(df, desiredName)
    return df

# left foot crosses
def LeftFootCross(event_df):
    df = event_df[(event_df.subEventName == 'Cross') & (event_df.subEventDescription.str.contains('left_foot'))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='LeftFootCross')
#     df = transform_zonal(df, desiredName)
    return df

# cross key passes
def CrossKeyPasses(event_df):
    df = event_df[(event_df.subEventName == 'Cross') & (event_df.subEventDescription.str.contains('key_pass'))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='CrossKeyPass')
#     df = transform_zonal(df, desiredName)
    return df

# accurate crosses
def AccurateCrosses(event_df):
    df = event_df[(event_df.subEventName == 'Cross') & (event_df.Result=="Accurate")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='AccCrosses')
#     df = transform_zonal(df, desiredName)
    return df

# inaccurate crosses
def InaccurateCrosses(event_df):
    df = event_df[(event_df.subEventName == 'Cross') & (event_df.Result=="Inaccurate")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='InaccCrosses')
#     df = transform_zonal(df, desiredName)
    return df

# Accurate crosses into penalty box
def AccCrossesBox(event_df):
    df = event_df[(event_df.subEventName == 'Cross') 
               & (event_df.Result=="Accurate") 
               & (event_df.xEnd > 82)
               & (event_df.yEnd < 80)
               & (event_df.yEnd > 20)].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='AccCrossesIntoBox')
#     df = transform_zonal(df, desiredName)
    return df

# Inaccurate crosses into penalty box
def InaccCrossesBox(event_df):
    df = event_df[(event_df.subEventName == 'Cross') 
               & (event_df.Result=="Inaccurate") 
               & (event_df.xEnd > 82)
               & (event_df.yEnd < 80)
               & (event_df.yEnd > 20)].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='InaccCrossesIntoBox')
#     df = transform_zonal(df, desiredName)
    return df



#### Tackles & Ground Duels

In [ ]:
# Sliding tackles won
def slidingTacklesWon(event_df):
    df = event_df[(event_df.subEventDescription.str.contains("sliding_tackle won"))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='SlidingTacklesWon')
#     df = transform_zonal(df, desiredName)
    return df

# Sliding tackles lost
def slidingTacklesLost(event_df):
    df = event_df[(event_df.subEventDescription.str.contains("sliding_tackle lost"))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='SlidingTacklesLost')
#     df = transform_zonal(df, desiredName)
    return df

# Ground Defending Duels Won
def GroundDuelsWon(event_df):
    df = event_df[(event_df.subEventName=="Ground defending duel") & event_df.subEventDescription.str.contains("won")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='GroundDefDuelsWon')
#     df = transform_zonal(df, desiredName)
    return df

# Ground Defending Duels Lost
def GroundDuelsLost(event_df):
    df = event_df[(event_df.subEventName=="Ground defending duel") & event_df.subEventDescription.str.contains("lost")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='GroundDefDuelsLost')
#     df = transform_zonal(df, desiredName)
    return df

# Ground Defensive Anticipations
def GroundDefAnticipations(event_df):
    df = event_df[(event_df.subEventName=="Ground defending duel") & event_df.subEventDescription.str.contains("anticipated")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='DefensiveAnticipations')
#     df = transform_zonal(df, desiredName)
    return df

# Ground Loose Balls Won
def GroundLooseBallsWon(event_df):
    df = event_df[(event_df.subEventName == 'Ground loose ball duel') &(event_df.subEventDescription.str.contains('won'))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='GroundLooseBallsWon')
#     df = transform_zonal(df, desiredName)
    return df

# Ground Loose Balls Lost
def GroundLooseBallsLost(event_df):
    df = event_df[(event_df.subEventName == 'Ground loose ball duel') &(event_df.subEventDescription.str.contains('lost'))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='GroundLooseBallsLost')
#     df = transform_zonal(df, desiredName)
    return df

# Ground attacking duels won
def GroundAttackDuelsWon(event_df):
    df = event_df[(event_df.subEventName == 'Ground attacking duel') &(event_df.subEventDescription.str.contains('won'))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='GroundAttackDuelsWon')
#     df = transform_zonal(df, desiredName)
    return df

# Ground attacking duels lost
def GroundAttackDuelsLost(event_df):
    df = event_df[(event_df.subEventName == 'Ground attacking duel') &(event_df.subEventDescription.str.contains('lost'))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='GroundAttackDuelsLost')
#     df = transform_zonal(df, desiredName)
    return df

# Offensive Touches in box
def OffTouchesBox(event_df):
    df = event_df[(event_df.subEventName == 'Touch')
              & (event_df.xStart > 82)
              & (event_df.yStart < 80)
              & (event_df.yStart > 20)].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='OffensiveTouchesBox')
#     df = transform_zonal(df, desiredName)
    return df


#### Shots & Goals

In [ ]:
# Shots on Target
def ShotsOnTarget(event_df):
    df = event_df[(event_df.subEventName=="Shot") & (event_df.Result=="Accurate")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='ShotsOnTarget')
#     df = transform_zonal(df, desiredName)
    return df

# Shots off Target
def ShotsOffTarget(event_df):
    df = event_df[(event_df.subEventName=="Shot") & (event_df.Result=="Inaccurate")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='ShotsOffTarget')
#     df = transform_zonal(df, desiredName)
    return df

# Shot opportunity
def shot_opportunities(event_df):
    df = event_df[(event_df.subEventName=="Shot") & (event_df.subEventDescription.str.contains("opportunity"))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='ShotOpportunities')
#     df = transform_zonal(df, desiredName)
    return df

# Accurate free kick shots
def accFreeKicks(event_df):
    df = event_df[(event_df.subEventName=="Free kick shot") & (event_df.Result=="Accurate")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='FreeKickShotAcc')
#     df = transform_zonal(df, desiredName)
    return df

# Inaccurate free kick shots
def InaccFreeKicks(event_df):
    df = event_df[(event_df.subEventName=="Free kick shot") & (event_df.Result=="Inaccurate")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='FreeKickShotInacc')
#     df = transform_zonal(df, desiredName)
    return df

# Did not score penalty
def PenaltyNotGoal(event_df):
    df = event_df[(event_df.subEventName=="Penalty") & (event_df.Result=="Inaccurate")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='PenaltyNotScored')
#     df = transform_zonal(df, desiredName)
    return df

# Goal scored with head/body
def HeaderGoal(event_df):
    df = event_df[(event_df.subEventName=="Shot") & (event_df.subEventDescription.str.match("goal "))
              & (event_df.subEventDescription.str.contains("head"))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='Header/Body_Goal')
#     df = transform_zonal(df, desiredName)
    return df    
    
# Goal scored with right foot
def RightFootGoal(event_df): 
    df = event_df[(event_df.subEventName=="Shot") & (event_df.subEventDescription.str.match("goal "))
              & (event_df.subEventDescription.str.contains("right_foot"))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='RightFootGoal')
#     df = transform_zonal(df, desiredName)
    return df

# Goal scored with left foot
def LeftFootGoal(event_df):
    df = event_df[(event_df.subEventName=="Shot") & (event_df.subEventDescription.str.match("goal "))
              & (event_df.subEventDescription.str.contains("left_foot"))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='LeftFootGoal')
#     df = transform_zonal(df, desiredName)
    return df

# Counter Attack Goal
def CounterAttackGoal(event_df):
    df = event_df[(event_df.subEventName=="Shot") & (event_df.subEventDescription.str.match("goal "))
              & (event_df.subEventDescription.str.contains("counter_attack"))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='CounterAttackGoal')
#     df = transform_zonal(df, desiredName)
    return df

# Interception Goal
def InterceptionGoal(event_df):
    df = event_df[(event_df.subEventName=="Shot") & (event_df.subEventDescription.str.match("goal "))
                & (event_df.subEventDescription.str.contains("interception"))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='InterceptionGoal')
#     df = transform_zonal(df, desiredName)
    return df

# Goal positions
def GoalPositions(event_df, pos):
    df = event_df[(event_df.subEventName=="Shot") & (event_df.subEventDescription.str.match("goal "))
                  & (event_df.subEventDescription.str.contains(pos))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='Goal_'+pos)
#     df = transform_zonal(df, desiredName)
    return df

def OwnGoals(event_df):
    df = event_df[event_df.subEventDescription.str.contains("own")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='OwnGoals')
#     df = transform_zonal(df, desiredName)
    return df

def shotAccuracy(event_df):
    df1 = event_df[(event_df.subEventName.str.contains('Shot')) & (event_df.Result=="Accurate")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='ShotAccuracy1')
    df2 = event_df[(event_df.subEventName.str.contains('Shot')) ].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='ShotAccuracy2')
    df1 = pd.merge(df1,df2, how="left", on=['playerId', 'matchId'])
    df1['shotAccuracy'] = df1['ShotAccuracy1'] / df1['ShotAccuracy2']
    df1.drop(['ShotAccuracy1', 'ShotAccuracy2'], inplace=True, axis =1)
#     df1 = transform_zonal(df, desiredName)
    return df1




In [ ]:
# # Get positional zones of goals for players

# possies = ['low_left', 'mid_left', 'high_left', 'low_center', 'mid_center', 'high_center', 'low_right', 'mid_right', 'high_right']
# for pos in possies:
#     a = GoalPositions(Events_England, pos)
#     b = GoalPositions(Events_France, pos)
#     c = GoalPositions(Events_Italy, pos)
#     d = GoalPositions(Events_Spain, pos)
#     e = GoalPositions(Events_Germany, pos)
#     df = pd.concat([a,b,c,d,e])
#     Player_Aggs = pd.merge(Player_Aggs, df, how= 'left', on =['playerId', 'matchId'])
#     Player_Aggs["Goal_"+pos] = Player_Aggs["Goal_"+pos].fillna(0)

# Player_Aggs = Player_Aggs.drop_duplicates()


####   Air duel

In [ ]:
def AerialDuelsWon(events_df):
    df = Events_England[(Events_England.subEventName=="Air duel") & (Events_England.subEventDescription.str.contains("won"))].groupby(['playerId', 'matchId', 'ZoneStart']).size().reset_index(name='AerialDuelsWon')
#     df = transform_zonal(df, desiredName)
    return df

def AerialDuelsLost(events_df):
    df =  Events_England[(Events_England.subEventName=="Air duel") & (Events_England.subEventDescription.str.contains("lost"))].groupby(['playerId', 'matchId', 'ZoneStart']).size().reset_index(name='AerialDuelsLost')
#     df = transform_zonal(df, desiredName)
    return df


#### Passing

In [ ]:
# Accurate forward passes
def acc_forward_pass(event_df):
    df = event_df[(event_df.subEventName=="Simple pass") 
                   & (event_df.attackMetres>0)
                   & (event_df.Result=="Accurate")
                   & (event_df.subEventDescription.str.match('generic play'))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='AccurateForwardPasses')
#     df = transform_zonal(df, desiredName)
    return df

# Inaccurate forward passes 
def inacc_forward_passes(event_df):
    df = event_df[(event_df.subEventName=="Simple pass") 
                   & (event_df.attackMetres>0)
                   & (event_df.Result=="Inaccurate")
                   & (event_df.subEventDescription.str.match('generic play'))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='InaccurateForwardPasses')
#     df = transform_zonal(df, desiredName)
    return df

# SuccessfulInterceptions ## when a player intercepts a play and makes a successful subsequent play
def success_intercept(event_df):
    df = event_df[(event_df.Result == 'Accurate')
                   & (event_df.subEventDescription.str.match('interception'))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='SuccessfulInterceptions')
#     df = transform_zonal(df, desiredName)
    return df

# CounterAttackInterceptions
def CounterAttackIntercepts(event_df):
    df = event_df[(event_df.subEventDescription.str.match('counter_attack interception'))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='CounterAttackIntercept')
#     df = transform_zonal(df, desiredName)
    return df

# DangerousBallLostPassing
def dangerous_ball_lost(event_df):
    df = event_df[(event_df.subEventName.str.contains("pass")) 
                   & (event_df.subEventDescription.str.match('dangerous_ball_lost'))].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='PassLostDangerous')
#     df = transform_zonal(df, desiredName)
    return df

# Counter attack involvements
def CounterAttackInvolvements(event_df):
    df = event_df[(event_df.subEventDescription.str.match('counter_attack')) 
                  & (event_df.Result == 'Accurate') ].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='CounterAttackInvolvements')
#     df = transform_zonal(df, desiredName)
    return df

# Total Assists
def total_assists(event_df):
    df = event_df[(event_df.subEventDescription.str.contains('assist')) 
                  & (event_df.Result == 'Accurate') ].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='TotalAssists')
#     df = transform_zonal(df, desiredName)
    return df

# Inaccurate Keypasses
def inacc_key_passes(event_df):
    df = event_df[(event_df.subEventDescription.str.contains('key_pass')) 
                  & (event_df.Result == "Inaccurate") ].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='InaccurateKeyPasses')
#     df = transform_zonal(df, desiredName)
    return df

# Accurate Keypasses
def acc_keypasses(event_df):
    df = event_df[(event_df.subEventDescription.str.contains('key_pass')) 
                  & (event_df.Result == "Accurate") ].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='AccurateKeyPasses')
#     df = transform_zonal(df, desiredName)
    return df

# Accurate Through Passes
def acc_throughs(event_df):
    df = event_df[(event_df.subEventDescription.str.contains('through')) 
                  & (event_df.Result == "Accurate") ].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='AccurateThroughPasses')
#     df = transform_zonal(df, desiredName)
    return df
# Inaccurate Through Passes
def inacc_throughs(event_df):
    df = event_df[(event_df.subEventDescription.str.contains('through')) 
                  & (event_df.Result == "Inaccurate") ].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='InaccurateThroughPasses')
#     df = transform_zonal(df, desiredName)
    return df

# Accurate Smart Passes 
def acc_smart_passes(event_df):
    df = event_df[(event_df.subEventName.str.match('Smart pass')) 
                  & (event_df.Result == "Accurate") ].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='AccurateSmartPasses')
#     df = transform_zonal(df, desiredName)
    return df

# Inaccurate Smart Passes 
def inacc_smartPasses(event_df):
    df = event_df[(event_df.subEventName.str.match('Smart pass')) 
                  & (event_df.Result == "Inaccurate") ].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='InaccurateSmartPasses')
#     df = transform_zonal(df, desiredName)
    return df

# Through passes into final third
def throughs_into_final(event_df):
    df = event_df[(event_df.subEventDescription.str.contains('through')) 
                  & (event_df.xStart < 67 ) 
                  & (event_df.xEnd > 66 ) 
                 & (event_df.Result == "Accurate") ].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='AccThroughsIntoFinalThird')
#     df = transform_zonal(df, desiredName)
    return df

# Accurate Passes into final third
def accPasses_into_final(event_df):
    df = event_df[ ( event_df.subEventName.str.contains('pass'))
        & (event_df.xStart < 67 ) 
                  & (event_df.xEnd > 66) 
                 & (event_df.Result == "Accurate")
                & (event_df.Goal_Value != 1)].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='AccPassesIntoFinalThird')
#     df = transform_zonal(df, desiredName)
    return df

# Accurate Passes into final third
def InaccPasses_into_final(event_df):
    df = event_df[ ( event_df.subEventName.str.contains('pass'))
        & (event_df.xStart < 67 ) 
                  & (event_df.xEnd > 66) 
                 & (event_df.Result == "Inaccurate")
                & (event_df.Goal_Value != 1)].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='InaccPassesIntoFinalThird')
#     df = transform_zonal(df, desiredName)
    return df

# Accurate Passes within final third
def accPasses_within_finalThird(event_df):
    df = event_df[ (event_df.subEventName.str.contains("pass") )
                   & (event_df.xStart > 66 ) 
                  & (event_df.xEnd > 66 ) 
                 & (event_df.Result == "Accurate")
                & (event_df.Goal_Value != 1)].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='AccPassesWithinFinalThird')
#     df = transform_zonal(df, desiredName)
    return df

# Inaccurate Passes within final third   
def inaccPasses_within_finalThird(event_df):
    df = event_df[ (event_df.subEventName.str.contains("pass") )
                   & (event_df.xStart > 66 ) 
                  & (event_df.xEnd > 66) 
                 & (event_df.Result == "Inaccurate")
                & (event_df.Goal_Value != 1)].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='InaccPassesWithinFinalThird')
#     df = transform_zonal(df, desiredName)
    return df


In [ ]:
def shortLongPassRatio(event_df):
    event_df['lateralMetres'] = event_df['yEnd']- event_df['yStart']
    df1 = event_df[(event_df.subEventName.str.contains('pass')) & ((event_df.attackMetres)<20) & (abs(event_df.lateralMetres<20)) & (event_df.Result=="Accurate")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='PassingAccuracy1')
    df2 = event_df[(event_df.subEventName.str.contains('pass')) & (((event_df.attackMetres)>20) | (abs(event_df.lateralMetres>20))) & (event_df.Result=="Accurate")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='PassingAccuracy2')
    df1 = pd.merge(df1,df2, how="left", on=['playerId', 'matchId'])
    df1['shortLongPassRatio'] = df1['PassingAccuracy1'] / df1['PassingAccuracy2']
    df1.drop(['PassingAccuracy1', 'PassingAccuracy2'], inplace=True, axis =1)
#     df1 = transform_zonal(df, desiredName)
    return df1

def longPassAccuracy(event_df):
    event_df['lateralMetres'] = event_df['yEnd']- event_df['yStart']
    df1 = event_df[(event_df.subEventName.str.contains('pass')) & ((abs(event_df.attackMetres)>20) | abs(event_df.lateralMetres>20)) & (event_df.Result=="Accurate")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='PassingAccuracy1')
    df2 = event_df[(event_df.subEventName.str.contains('pass')) & ((abs(event_df.attackMetres)>20) | abs(event_df.lateralMetres>20)) ].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='PassingAccuracy2')
    df1 = pd.merge(df1,df2, how="left", on=['playerId', 'matchId'])
    df1['longPassAccuracy'] = df1['PassingAccuracy1'] / df1['PassingAccuracy2']
    df1.drop(['PassingAccuracy1', 'PassingAccuracy2'], inplace=True, axis =1)
#     df1 = transform_zonal(df, desiredName)
    return df1

def shortPassAccuracy(event_df):
    event_df['lateralMetres'] = event_df['yEnd']- event_df['yStart']
    df1 = event_df[(event_df.subEventName.str.contains('pass')) & (abs(event_df.attackMetres)<20) & (abs(event_df.lateralMetres<20)) & (event_df.Result=="Accurate")].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='PassingAccuracy1')
    df2 = event_df[(event_df.subEventName.str.contains('pass'))& (abs(event_df.attackMetres)<20) & (abs(event_df.lateralMetres<20))  ].groupby(['playerId', 'matchId', "ZoneStart"]).size().reset_index(name='PassingAccuracy2')
    df1 = pd.merge(df1,df2, how="left", on=['playerId', 'matchId'])
    df1['shortPassAccuracy'] = df1['PassingAccuracy1'] / df1['PassingAccuracy2']
    df1.drop(['PassingAccuracy1', 'PassingAccuracy2'], inplace=True, axis =1)
    return df1

In [ ]:
Events_England["ReceivingPlayer"] = Events_England['playerId'].shift(-1)

#### Live match analysis

In [ ]:
liveTimes = []
for time in range(600, 6000, 600):
    liveTimes.append(time)

In [ ]:
#### path = os.getcwd()
# Events_England.to_csv(path+'/Events_England_processed')
# Events_France.to_csv(path+'/Events_France_processed')
# Events_Italy.to_csv(path+'/Events_Italy_processed')
# Events_Spain.to_csv(path+'/Events_Spain_processed')
# Events_Germany.to_csv(path+'/Events_Germany_processed')


In [ ]:

# Events_England_live = Events_England[Events_England.eventSec <= liveTime]
# Events_France_live = Events_France[Events_France.eventSec <= liveTime]
# Events_Italy_live = Events_Italy[Events_Italy.eventSec <= liveTime]
# Events_Spain_live = Events_Spain[Events_Spain.eventSec <= liveTime]
# Events_Germany_live = Events_Germany[Events_Germany.eventSec <= liveTime]


In [ ]:

Player_Aggs_live = Players[['shortName','Position', 'playerId','weight','height',  'foot' ]]
Player_Aggs_live = pd.merge(Player_Aggs_live, position_df, how = 'right', on= 'playerId').sort_values('matchId')
Player_Aggs_live = pd.merge(Player_Aggs_live, p_refs, on = ['matchId', 'playerId'])
Player_Aggs_live = pd.merge(Player_Aggs_live, Teams[['teamId', 'name']], on = 'teamId')
Player_Aggs_live = pd.merge(Player_Aggs_live, Matches[['matchId', 'homeTeamId', 'awayTeamId']], on = 'matchId')
Player_Aggs_live.loc[Player_Aggs_live.teamId == Player_Aggs_live.homeTeamId, 'homeAway'] = "home"
Player_Aggs_live.loc[Player_Aggs_live.teamId == Player_Aggs_live.awayTeamId, 'homeAway'] = "away"


In [ ]:
def playerGameTime_live(event_df, time):
    df = event_df.groupby(['matchId','playerId'])['eventSec'].agg([ 'min']).reset_index()
    df['liveTime'] = time
    df['gameTime (min)'] = round((df['liveTime'] - df['min'])/60) # game time to the nearest minute
    # Classify Match Result as W, L or D for teams
    conditions = [
        (df['gameTime (min)']  < 1),
        (df['gameTime (min)'] > 0)
        ]

    # create a list of the values we want to assign for each condition
    values = [0, df['gameTime (min)'].max()]

    # create a new column and use np.select to assign values to it using our lists as arguments
    df['gameTime (min)'] = np.select(conditions, values)
    
    df.drop(['liveTime', 'min'], inplace=True, axis=1)
    return df



In [ ]:
# convenience function for event_df parsing
def parseEvent_df_live(func, new_feature, time, df_England_live, df_France_live, df_Italy_live, df_Spain_live, df_Germany_live):
    
    aa = func(df_England_live)
    aa = transform_zonal(aa, new_feature)
    bb = func(df_France_live)
    bb = transform_zonal(bb, new_feature)
    cc = func(df_Italy_live)
    cc = transform_zonal(cc, new_feature)
    dd = func(df_Spain_live)
    dd = transform_zonal(dd, new_feature)
    ee = func(df_Germany_live)
    ee = transform_zonal(ee, new_feature)
    
    
    try:
        return pd.concat([aa,bb,cc,dd,ee])
    except:
        return None


In [ ]:
# function for adding new feature column to Player_Aggs df

def alter_Player_Aggs_live(new_feature_function, new_feature, PA_df, df_England_live, df_France_live, df_Italy_live, df_Spain_live, df_Germany_live):

    df = parseEvent_df_live(new_feature_function, new_feature, time, df_England_live, df_France_live, df_Italy_live, df_Spain_live, df_Germany_live)

    try:
        PA_df = pd.merge(PA_df, df, how= 'left', on =['playerId', 'matchId'])
    
    except:
        return PA_df
    
    return PA_df


In [ ]:
Player_Aggs_live = Players[['shortName','Position', 'playerId','weight','height',  'foot' ]]
Player_Aggs_live = pd.merge(Player_Aggs_live, position_df, how = 'right', on= 'playerId').sort_values('matchId')
Player_Aggs_live = pd.merge(Player_Aggs_live, p_refs, on = ['matchId', 'playerId'])
Player_Aggs_live = pd.merge(Player_Aggs_live, Teams[['teamId', 'name']], on = 'teamId')
Player_Aggs_live = pd.merge(Player_Aggs_live, Matches[['matchId', 'homeTeamId', 'awayTeamId']], on = 'matchId')
Player_Aggs_live.loc[Player_Aggs_live.teamId == Player_Aggs_live.homeTeamId, 'homeAway'] = "home"
Player_Aggs_live.loc[Player_Aggs_live.teamId == Player_Aggs_live.awayTeamId, 'homeAway'] = "away"

In [ ]:
# funkies = [AccBackwardMetres, InaccBackwardMetres, AccForwardMetres, InaccForwardMetres, SimulationFouls, Fouls, Clearances, AccLaunchMetres, InaccLaunchMetres, FreeKickCrossKey, FreeKickCrossAccuracy, dangerousOpponentHalfRecoveries, dangerousOwnHalfBallLost, dangerousDefDuelsLost, Saves, LeavingLine, SuccessAccelerations, FailedAccelerations, AccelDistance, AccelsIntoFinalThird, AccelsWithinFinalThird, RightFootCross, LeftFootCross, CrossKeyPasses, AccurateCrosses, InaccurateCrosses, AccCrossesBox, InaccCrossesBox, slidingTacklesWon, slidingTacklesLost, GroundDuelsWon, GroundDuelsLost, GroundDefAnticipations, GroundLooseBallsWon, GroundLooseBallsLost, GroundAttackDuelsWon, GroundAttackDuelsLost, OffTouchesBox, ShotsOnTarget, ShotsOffTarget, shot_opportunities, accFreeKicks, InaccFreeKicks, PenaltyNotGoal, AerialDuelsWon, AerialDuelsLost, inaccPasses_within_finalThird, accPasses_within_finalThird, accPasses_into_final, InaccPasses_into_final, throughs_into_final, inacc_smartPasses, acc_smart_passes, inacc_throughs, acc_throughs, acc_keypasses, inacc_key_passes, CounterAttackInvolvements, dangerous_ball_lost, CounterAttackIntercepts, success_intercept, inacc_forward_passes, acc_forward_pass, corner_assists, corner_opportunity, corner_success, corner_fail]
# var_noms = ["AccBackMetres",  "InaccBackMetres",  "AccForwardMetres",  "InaccForwardMetres",  "SimulationFouls",  "FoulsCommited",  "Clearances",  "AccLaunchMetres",  "InaccLaunchMetres",  "FKKeyCross",  "AccFreeKickCrosses",  "dangerousOpponentHalfRecoveries",  "dangerousOwnHalfBallLost",  "dangerousDefDuelsLost",  "GkSaves", "GKLeavingLineInstance",  "SuccessfulAccels",  "FailedAccels",  "CumAccelerationDist",  "AccelsDistIntoFinal3rd",  "AccelsDistWithinFinal3rd",  "RightFootCross",  "LeftFootCross",  "CrossKeyPass",  "AccCrosses",  "InaccCrosses",  "AccCrossesIntoBox",  "InaccCrossesIntoBox",  "SlidingTacklesWon",  "SlidingTacklesLost",  "GroundDefDuelsWon",  "GroundDefDuelsLost",  "DefensiveAnticipations",  "GroundLooseBallsWon",  "GroundLooseBallsLost",  "GroundAttackDuelsWon",  "GroundAttackDuelsLost",  "OffensiveTouchesBox",  "ShotsOnTarget",  "ShotsOffTarget",  "ShotOpportunities",  "FreeKickShotAcc",  "FreeKickShotInacc",  "PenaltyNotScored",  "AerialDuelsWon",  "AerialDuelsLost",  "InaccPassesWithinFinalThird",  "AccPassesWithinFinalThird",  "AccPassesIntoFinalThird",  "InaccPassesIntoFinalThird",  "AccThroughsIntoFinalThird",  "InaccurateSmartPasses",  "AccurateSmartPasses",  "InaccurateThroughPasses",  "AccurateThroughPasses",  "AccurateKeyPasses",  "InaccurateKeyPasses",  "CounterAttackInvolvements",  "PassLostDangerous",  "CounterAttackIntercept",  "SuccessfulInterceptions",  "InaccurateForwardPasses",  "AccurateForwardPasses",  "CornerAssists",  "CornerOpportunitiesCreated",  "SuccessfulCorners",  "FailedCorners"]
    
    
# for indx in range(len(funkies)):
#     Player_Aggs2 = alter_Player_Aggs(funkies[indx], var_noms[indx], Player_Aggs2)
#     Player_Aggs2 = Player_Aggs2.drop_duplicates()
#     remove_col = [col for col in Player_Aggs2 if col.endswith("_0")]
#     if remove_col:
#         Player_Aggs2.drop(columns = remove_col, inplace=True, axis=1)

# Player_Aggs2 = Player_Aggs2.loc[:, (Player_Aggs2 != 0).any(axis=0)]


In [ ]:
# # # Live Match # # #

def get_live_players(player_aggs_live, time):
    
    # derive gameTime per player and store in summary table
    a = playerGameTime_live(Events_England, time)
    b = playerGameTime_live(Events_France, time)
    c = playerGameTime_live(Events_Italy, time)
    d = playerGameTime_live(Events_Spain, time)
    e = playerGameTime_live(Events_Germany, time)

    playingTime_live = pd.concat([a,b,c,d,e])

    player_aggs_live = pd.merge(player_aggs_live, playingTime_live, how = 'left', on = ['playerId', 'matchId'])

    funkies = [AccBackwardMetres, InaccBackwardMetres, AccForwardMetres, InaccForwardMetres, SimulationFouls, Fouls, Clearances, AccLaunchMetres, InaccLaunchMetres, FreeKickCrossKey, FreeKickCrossAccuracy, dangerousOpponentHalfRecoveries, dangerousOwnHalfBallLost, dangerousDefDuelsLost, Saves, LeavingLine, SuccessAccelerations, FailedAccelerations, AccelDistance, AccelsIntoFinalThird, AccelsWithinFinalThird, RightFootCross, LeftFootCross, CrossKeyPasses, AccurateCrosses, InaccurateCrosses, AccCrossesBox, InaccCrossesBox, slidingTacklesWon, slidingTacklesLost, GroundDuelsWon, GroundDuelsLost, GroundDefAnticipations, GroundLooseBallsWon, GroundLooseBallsLost, GroundAttackDuelsWon, GroundAttackDuelsLost, OffTouchesBox, ShotsOnTarget, ShotsOffTarget, shot_opportunities, accFreeKicks, InaccFreeKicks, PenaltyNotGoal, AerialDuelsWon, AerialDuelsLost, inaccPasses_within_finalThird, accPasses_within_finalThird, accPasses_into_final, InaccPasses_into_final, throughs_into_final, inacc_smartPasses, acc_smart_passes, inacc_throughs, acc_throughs, acc_keypasses, inacc_key_passes, CounterAttackInvolvements, dangerous_ball_lost, CounterAttackIntercepts, success_intercept, inacc_forward_passes, acc_forward_pass, corner_assists, corner_opportunity, corner_success, corner_fail]
    var_noms = ["AccBackMetres",  "InaccBackMetres",  "AccForwardMetres",  "InaccForwardMetres",  "SimulationFouls",  "FoulsCommited",  "Clearances",  "AccLaunchMetres",  "InaccLaunchMetres",  "FKKeyCross",  "AccFreeKickCrosses",  "dangerousOpponentHalfRecoveries",  "dangerousOwnHalfBallLost",  "dangerousDefDuelsLost",  "GkSaves", "GKLeavingLineInstance",  "SuccessfulAccels",  "FailedAccels",  "CumAccelerationDist",  "AccelsDistIntoFinal3rd",  "AccelsDistWithinFinal3rd",  "RightFootCross",  "LeftFootCross",  "CrossKeyPass",  "AccCrosses",  "InaccCrosses",  "AccCrossesIntoBox",  "InaccCrossesIntoBox",  "SlidingTacklesWon",  "SlidingTacklesLost",  "GroundDefDuelsWon",  "GroundDefDuelsLost",  "DefensiveAnticipations",  "GroundLooseBallsWon",  "GroundLooseBallsLost",  "GroundAttackDuelsWon",  "GroundAttackDuelsLost",  "OffensiveTouchesBox",  "ShotsOnTarget",  "ShotsOffTarget",  "ShotOpportunities",  "FreeKickShotAcc",  "FreeKickShotInacc",  "PenaltyNotScored",  "AerialDuelsWon",  "AerialDuelsLost",  "InaccPassesWithinFinalThird",  "AccPassesWithinFinalThird",  "AccPassesIntoFinalThird",  "InaccPassesIntoFinalThird",  "AccThroughsIntoFinalThird",  "InaccurateSmartPasses",  "AccurateSmartPasses",  "InaccurateThroughPasses",  "AccurateThroughPasses",  "AccurateKeyPasses",  "InaccurateKeyPasses",  "CounterAttackInvolvements",  "PassLostDangerous",  "CounterAttackIntercept",  "SuccessfulInterceptions",  "InaccurateForwardPasses",  "AccurateForwardPasses",  "CornerAssists",  "CornerOpportunitiesCreated",  "SuccessfulCorners",  "FailedCorners"]
    
    Events_England_live = Events_England[Events_England.eventSec <= time]
    Events_France_live = Events_France[Events_France.eventSec <= time]
    Events_Italy_live = Events_Italy[Events_Italy.eventSec <= time]
    Events_Spain_live = Events_Spain[Events_Spain.eventSec <= time]
    Events_Germany_live = Events_Germany[Events_Germany.eventSec <= time]
    
    for indx in range(len(funkies)):
        player_aggs_live = alter_Player_Aggs_live(funkies[indx], var_noms[indx], player_aggs_live, Events_England_live, Events_France_live, Events_Italy_live, Events_Spain_live, Events_Germany_live)
        player_aggs_live = player_aggs_live.drop_duplicates()
        remove_col = [col for col in player_aggs_live if col.endswith("_0")]
        if remove_col:
            player_aggs_live.drop(columns = remove_col, inplace=True, axis=1)  

    player_aggs_live.iloc[:,15:] = player_aggs_live.iloc[:,15:].fillna(0)
    player_aggs_live.iloc[:,15:] = player_aggs_live.iloc[:,15:].div(player_aggs_live["gameTime (min)"], axis =0).fillna(0)*90
    player_aggs_live = player_aggs_live.loc[:, (player_aggs_live != 0).any(axis=0)]
    
    return player_aggs_live


In [ ]:
# Player_Aggs_live10 = get_live_players(Player_Aggs_live, 600)
# Player_Aggs_live20 = get_live_players(Player_Aggs_live, 1200)
# Player_Aggs_live30 = get_live_players(Player_Aggs_live, 1800)
# Player_Aggs_live40 = get_live_players(Player_Aggs_live, 2400)
# Player_Aggs_live50 = get_live_players(Player_Aggs_live, 3000)
# Player_Aggs_live60 = get_live_players(Player_Aggs_live, 3600)
# Player_Aggs_live70 = get_live_players(Player_Aggs_live, 4200)
# Player_Aggs_live80 = get_live_players(Player_Aggs_live, 4800)
# Player_Aggs_live90 = get_live_players(Player_Aggs_live, 5400)



In [ ]:
# os.chdir('/group/interns202010/jmakins/Data/events')


In [ ]:
os.chdir('/kaggle/input/youwot')

In [ ]:
# import zipfile
# with zipfile.ZipFile('/kaggle/input/youwot', 'r') as zip_ref:
#     zip_ref.extractall('/kaggle/input/youwot')

In [ ]:
# path = os.getcwd()

# Player_Aggs_live10.to_csv(path+'/Player_Aggs_live10')
# Player_Aggs_live20.to_csv(path+'/Player_Aggs_live20')
# Player_Aggs_live30.to_csv(path+'/Player_Aggs_live30')
# Player_Aggs_live40.to_csv(path+'/Player_Aggs_live40')
# Player_Aggs_live50.to_csv(path+'/Player_Aggs_live50')
# Player_Aggs_live60.to_csv(path+'/Player_Aggs_live60')
# Player_Aggs_live70.to_csv(path+'/Player_Aggs_live70')
# Player_Aggs_live80.to_csv(path+'/Player_Aggs_live80')
# Player_Aggs_live90.to_csv(path+'/Player_Aggs_live90')

# Player_Aggs_live10 = pd.read_csv('Player_Aggs_live10')
# Player_Aggs_live10.drop(["Unnamed: 0"], axis=1, inplace=True)
# Player_Aggs_live20= pd.read_csv('Player_Aggs_live20')
# Player_Aggs_live20.drop(["Unnamed: 0"], axis=1, inplace=True)
# Player_Aggs_live30= pd.read_csv('Player_Aggs_live30')
# Player_Aggs_live30.drop(["Unnamed: 0"], axis=1, inplace=True)
# Player_Aggs_live40= pd.read_csv('Player_Aggs_live40')
# Player_Aggs_live40.drop(["Unnamed: 0"], axis=1, inplace=True)
# Player_Aggs_live50= pd.read_csv('Player_Aggs_live50')
# Player_Aggs_live50.drop(["Unnamed: 0"], axis=1, inplace=True)
# Player_Aggs_live60= pd.read_csv('Player_Aggs_live60')
# Player_Aggs_live60.drop(["Unnamed: 0"], axis=1, inplace=True)
# Player_Aggs_live70= pd.read_csv('Player_Aggs_live70')
# Player_Aggs_live70.drop(["Unnamed: 0"], axis=1, inplace=True)
# Player_Aggs_live80= pd.read_csv('Player_Aggs_live80')
# Player_Aggs_live80.drop(["Unnamed: 0"], axis=1, inplace=True)
# Player_Aggs_live90= pd.read_csv('Player_Aggs_live90')
# Player_Aggs_live90.drop(["Unnamed: 0"], axis=1, inplace=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc
from sklearn.cluster import KMeans 
from sklearn import metrics 
from scipy.spatial.distance import cdist 
import matplotlib.pyplot as plt  
import matplotlib.patches as patches
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle
from sklearn import metrics
from random import seed

import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout

In [ ]:
del Player_Aggs_live10

In [ ]:
Player_Aggs_live90= pd.read_csv('Player_Aggs_live90')
Player_Aggs_live90.drop(["Unnamed: 0"], axis=1, inplace=True)

# Events_England = pd.read_csv('eventsEvents_England_processed')
# Events_Italy = pd.read_csv('eventsEvents_Italy_processed')
# Events_Germany = pd.read_csv('eventsEvents_Germany_processed')
# Events_France = pd.read_csv('eventsEvents_France_processed')
# Events_Spain = pd.read_csv('eventsEvents_Spain_processed')

### fDNN

In [ ]:
def Matches_df_fDNN(TAM):
    df2 = TAM.groupby('matchId').agg(list).reset_index()
    df2.drop(["Result","homeTeamId", "awayTeamId", "height", "weight", 'playerId', "xStart", "yStart", "gameTime (min)"], inplace=True, axis=1)

    df4 = pd.concat([df2, pd.DataFrame(df2['teamId'].to_list(), columns=['team1','team2'])], axis=1, sort = False)
    df4.drop(["teamId"], inplace=True, axis = 1)
    result = pd.merge(df4, Matches[['matchId', "homeTeamId", "awayTeamId"]], on='matchId', how='inner')


    df3 = result.iloc[:,-4:]
    for var in list(result.columns)[1:-4]:

        df3 = pd.concat([df3, pd.DataFrame(result[var].to_list(), columns=["1_" + var, "2_" + var])], axis=1, sort = False)

        # Correctly apply allocate home and away aggregate statistics to correct teams
        df3['home'+var] = (
            np.select(
                condlist=[result['team1'] == result['homeTeamId'], result['team2'] == result['homeTeamId']], 
                choicelist=[df3["1_" + var], df3["2_" + var]]))

        df3['away'+var] = (
            np.select(
                condlist=[result['team1'] == result['awayTeamId'], result['team2'] == result['awayTeamId']], 
                choicelist=[df3["1_" + var], df3["2_" + var]]))

        # # drop useless columns that have been replaced
        df3.drop([ "1_" + var, "2_" + var], inplace = True, axis = 1 )

    df3 = pd.concat([result[['matchId']],df3], axis=1)
    df3 = pd.merge(df3, Matches[["matchId", "Result"]], how ="inner", on='matchId')
    df3.drop(["team1", "team2"], axis=1, inplace=True)

    return df3


In [ ]:
Team_Aggs_Matches = Player_Aggs_live90.groupby(['matchId', 'teamId' ]).sum().reset_index()
Team_Aggs_Matches = pd.merge(Team_Aggs_Matches, Matches[['matchId', "Result"]], how = 'left', on = 'matchId' )
result = Matches_df_fDNN(Team_Aggs_Matches)

In [ ]:
X_all = result.loc[:, ~result.columns.isin(["Result",'teamId', 'matchId', 'homeTeamId', 'awayTeamId', 'gameTime (min)'])]
y_all = result["Result"].values
X_all = X_all.replace([np.inf, -np.inf], np.nan)
X_all = X_all.fillna(X_all.mean())
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, shuffle=True, test_size= 0.2, train_size=0.8, random_state=3)


In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:

clf = RandomForestClassifier(n_estimators = 500, random_state=123, max_depth = 5)
clf.fit(X_train, y_train)


In [ ]:
labels = [-1,0,1]
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, labels = labels))
print('accuracy score: {0:.4f}'.format(accuracy_score(y_test,y_pred)))

In [ ]:
print(pd.crosstab(y_test, y_pred, rownames=['Actual Result'], colnames=['Predicted Result']))
class_name = list(set(y_test))
print(class_name)

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (16, 6)

In [ ]:
df_feature_importance = pd.DataFrame(clf.feature_importances_, index=X_all.columns, columns=['feature importance']).sort_values('feature importance', ascending=False)


In [ ]:
df_feature_importance[0:20].plot(kind='bar')



In [ ]:
## DNN

In [ ]:
# X_trainD, X_testD, y_trainD, y_testD = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
# from sklearn.preprocessing import StandardScaler

In [ ]:
# scaler = StandardScaler().fit(X_trainD)

# X_trainD = scaler.transform(X_trainD)

# X_testD = scaler.transform(X_testD)

# X_testD2 = scaler.transform(X_test)

In [ ]:
# # Define per-fold score containers
# acc_per_fold = []
# loss_per_fold = []

# kfold = KFold(n_splits=5, shuffle=True)

# fold_no=1

# for train, test in kfold.split(X_trainD, y_trainD):
    
#     dnn = Sequential()
#     #dnn.add(Flatten(input_shape=(500,3)))
#     #dnn.add(Dense(512, activation='relu',kernel_initializer = 'he_normal'))
#     #dnn.add(Dropout(0.3))
#     dnn.add(Dense(256, activation='relu', kernel_initializer = 'he_normal' ,input_shape=(X_testD2.shape[1],)))
#     dnn.add(Dropout(0.4))
#     dnn.add(Dense(128, activation='relu',kernel_initializer = 'he_normal'))
#     dnn.add(Dropout(0.4))
#     dnn.add(Dense(64, activation='relu',kernel_initializer = 'he_normal'))
#     dnn.add(Dropout(0.4))
#     dnn.add(Dense(1, activation='sigmoid',kernel_initializer = 'he_normal'))
#     #optim = keras.optimizers.SGD(lr=0.01, momentum=0.975, decay=2e-06, nesterov=True)
#     opt = keras.optimizers.Adam(learning_rate=0.00000001)


#     dnn.compile(loss='categorical_crossentropy',
#     optimizer=opt, metrics=['accuracy'])


#     #history = model.fit(X_train2, y_train2,
#     #batch_size = 32, epochs = 1,  verbose = 2)#, validation_data= (x_valid, y_valid))

#     history = dnn.fit(X_trainD[train], y_trainD[train], validation_data = (X_trainD[test], y_trainD[test]), epochs=2500, batch_size=32, verbose =2)

#     # batchsize 256
#     #lr 0.00001, 0.0001

#     print(dnn.summary())

#     # Generate generalization metrics
#     scores = model.evaluate(X_trainD[test], y_trainD[test], verbose=0)
#     print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
#     acc_per_fold.append(scores[1] * 100)
#     loss_per_fold.append(scores[0])
#     # Increase fold number
#     fold_no = fold_no + 1

# # score = model.evaluate(X_test2, y_test2, verbose=0)
# # print('Test loss:', score[0])
# # print('Test top 1 accuracy:', score[1])

# # == Provide average scores ==
# print('------------------------------------------------------------------------')
# print('Score per fold')
# for i in range(0, len(acc_per_fold)):
#     print('------------------------------------------------------------------------')
#     print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
# print('------------------------------------------------------------------------')
# print('Average scores for all folds:')
# print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
# print(f'> Loss: {np.mean(loss_per_fold)}')
# print('------------------------------------------------------------------------')

In [ ]:

# y_pred = dnn.predict(X_testD2)
# #Converting predictions to label
# pred = list()
# for i in range(len(y_pred)):
#     pred.append(np.argmax(y_pred[i]))
# # #Converting one hot encoded test label to label
# # test = list()
# # for i in range(len(y_test2)):
# #     test.append(np.argmax(y_test[i]))





In [ ]:
# a = accuracy_score(pred,y_test)
# print('Accuracy is:', a*100)

## fdnn

In [ ]:
X_test2 = [tree.predict(X_test) for tree in clf.estimators_]
X_test2 = np.transpose(X_test2)
X_train2 = [tree.predict(X_train) for tree in clf.estimators_]
X_train2 = np.transpose(X_train2)


In [ ]:
# x_total = [tree.predict(result) for tree in clf.estimators_]
# x_total = np.transpose(x_total)

In [ ]:
def X_transform(X):
    df = []  
    for sample in X:
        s = []
        for feature in sample:
            if feature == 2:
                s.append([0, 0, 1])
            elif feature == 1:
                s.append([0, 1, 0])
            else:
                s.append([1, 0, 0])
        df.append(s)
    df = np.array(df)
    return df

In [ ]:
X_train2 = X_transform(X_train2)
X_test2 = X_transform(X_test2)

In [ ]:
X_train2.shape
X_test2.shape

In [ ]:
## one-hot encode the labels
def y_transform(Y):
    df = []
    for l in Y:
        if l == 1:
            df.append([0, 0, 1])
        elif l == 0:
            df.append([0, 1, 0])
        else:
            df.append([1, 0, 0])
    df = np.array(df,dtype=int)
    return df

In [ ]:
y_train2 = y_transform(y_train)
y_test2 = y_transform(y_test)

In [ ]:
print(y_train2.shape)
print(y_test2.shape)

In [ ]:
# X_train2, X_test3, y_train2, y_test3 = train_test_split(X_train2, y_train2, test_size=0.2)


In [ ]:
print(y_train2.shape)
print(X_train2.shape)


##### Keras DNN

In [ ]:
# Model

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

kfold = KFold(n_splits=3, shuffle=True)

fold_no=1

for train, test in kfold.split(X_train2, y_train2):

    model = Sequential()
    model.add(Flatten(input_shape=(500,3)))
    model.add(Dense(256, activation='relu', kernel_initializer = 'he_normal' ,input_shape=(500*3,)))
    model.add(Dropout(0.4))
    model.add(Dense(128, activation='relu',kernel_initializer = 'he_normal'))
    model.add(Dropout(0.4))
    model.add(Dense(64, activation='relu',kernel_initializer = 'he_normal'))
    model.add(Dropout(0.4))
    model.add(Dense(3, activation='softmax',kernel_initializer = 'he_normal'))
    opt = keras.optimizers.Adam(learning_rate=0.000001)

    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    #history = model.fit(X_train2, y_train2,
    #batch_size = 32, epochs = 1,  verbose = 2)#, validation_data= (x_valid, y_valid))

    history = model.fit(X_train2[train], y_train2[train], validation_data = (X_train2[test], y_train2[test]), epochs=2000, batch_size=32, verbose =2)

    # batchsize 256
    #lr 0.00001, 0.0001

    print(model.summary())
    
    # Generate generalization metrics
    scores = model.evaluate(X_train2[test], y_train2[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    # Increase fold number
    fold_no = fold_no + 1

# score = model.evaluate(X_test2, y_test2, verbose=0)
# print('Test loss:', score[0])
# print('Test top 1 accuracy:', score[1])



In [ ]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [ ]:
y_pred = model.predict(X_test2)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(y_test2)):
    test.append(np.argmax(y_test2[i]))

In [ ]:
a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

In [ ]:
for i in range(len(pred)):
    if pred[i]==2:
        pred[i] = 1
    elif pred[i]==0:
        pred[i]= -1
    else:
        pred[i]=0

In [ ]:
for i in range(len(test)):
    if test[i]==2:
        test[i] = 1
    elif test[i]==0:
        test[i]= -1
    else:
        test[i]=0

In [ ]:
cf_matrix = confusion_matrix(y_test, pred)
import seaborn as sns
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams['font.size'] = 14
fig, ax = plt.subplots(1)
x_axis_labels,y_axis_labels = ["Loss", "Draw", "Win"], ["Loss", "Draw", "Win"]
#sns.color_palette("magma")
sns.heatmap(cf_matrix, annot=True, fmt="d", xticklabels=x_axis_labels, yticklabels=y_axis_labels, center=50, linecolor="black", cmap='flag', linewidths=0.2, annot_kws={'size':20},cbar=False), #,cmap="PiYG")
plt.xlabel("Predicted Result",  color = "white")
plt.ylabel("Actual Result",  color = "white")



In [ ]:
collections.Counter(test)

In [ ]:
labels = [-1,0,1]

print(classification_report(test, pred, labels = labels,target_names=['Loss', 'Draw', 'Win']))
print('accuracy score: {0:.4f}'.format(accuracy_score(test,pred)))

In [ ]:
print(pd.crosstab(np.array(test), np.array(pred), rownames=['Actual Result'], colnames=['Predicted Result']))
class_name = list(set(test))
print(class_name)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss']) 
plt.title('Model loss') 
plt.ylabel('Loss') 
plt.xlabel('Epoch') 
plt.legend(['Train', 'Test'], loc='upper left') 
plt.show()

###### XGBoost DNN

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0, 0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.2, 0.4, 0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5, 6, 7, 8],
        'learning_rate': [0.0001,0.001, 0.01, 0.02,0.1, 0.2, 0.3],
        'n_estimators': [100,200,500,1000]
        }

In [ ]:
import datetime
from datetime import *

In [ ]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# param_comb = 5
# folds = 5

# skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

# random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='accuracy', n_jobs=4, cv=skf.split(X_train,y_train), verbose=3, random_state=1001 )

# # Here we go
# start_time = timer(None) # timing starts from this point for "start_time" variable
# random_search.fit(X_train, y_train)
# timer(start_time) # timing ends here for "start_time" variable

# # xgb.fit(X_train, y_train)




In [ ]:
# print('\n All results:')
# print(random_search.cv_results_)
# print('\n Best estimator:')
# print(random_search.best_estimator_)
# print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
# print(random_search.best_score_ * 2 - 1)
# print('\n Best hyperparameters:')
# print(random_search.best_params_)

In [ ]:
##### CREATE XGB WITH OPTIMIZED PARAMS
# Best hyperparameters: {'subsample': 0.8, 'n_estimators': 500, 'min_child_weight': 5, 'max_depth': 7, 'learning_rate': 0.2, 'gamma': 1.5, 'colsample_bytree': 0.8}

xgb = XGBClassifier(features_names=X_all.columns, gamma=1.5, n_estimators =500, max_depth=7, min_child_weight=5)
xgb.fit(X_train, y_train)


In [ ]:
# feat_imp = pd.Series(xgb.booster().get_fscore()).sort_values(ascending=False)
# feat_imp.plot(kind='bar', title='Feature Importances')57.1
# plt.ylabel('Feature Importance Score')

In [ ]:
# make predictions for test data
y_pred = xgb.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


In [ ]:
from xgboost import plot_importance
# plot feature importance
plot_importance(xgb, max_num_features=25, ylabel="Features")
plt.figure(figsize=(5,5))
fig.set_size_inches(6.5, 4.5, forward=True)
plt.show()

In [ ]:
from xgboost import plot_tree
# plot single tree
plot_tree(xgb, num_trees=1, rankdir='LR')
#plt.rcParams['figure.figsize'] = [50, 20]
mpl.rcParams.update(mpl.rcParamsDefault)
plt.show()

In [ ]:
BREAK sfkldfkjsdkfl asap rocky

#### Result visualization

In [ ]:
names = ["RF", "fDNN", "xgboost"]
minutes = [10,20,30,40,50,60,70,80,90]
x = list(range(9))
fdnn = []
xb = []
rf = []

accuracies = [47.54, 47.55, 46.17, 50.55, 59.56, 46.72, 50.82, 54.10, 51.37, 51.91, 57.377, 57.10, 51.64, 54.92, 59.02, 53.01, 59.56, 52.8, 52.19, 57.38, 59.84, 53.55, 60.11, 63.93, 53.55, 64.481, 64.75]
for i in range(0,27,3):
    try:
        rf.append(accuracies[i])
        fdnn.append(accuracies[i+1])
        xb.append(accuracies[i+2])
    except:
        pass

In [ ]:
df = pd.DataFrame({'RF': rf, "fDNN": fdnn, "XGBoost": xb, "Minutes": minutes})

#plt.style.use('fivethirtyeight')
#plt.style.use('seaborn-darkgrid')
#my_dpi=96
fig = plt.figure(figsize=(480/my_dpi, 480/my_dpi), dpi=my_dpi)
fig.patch.set_facecolor('#241C24')


# create a color palette
palette = plt.get_cmap('Dark2')
 
# multiple line plot

plt.plot(df['Minutes'], df["RF"], marker='', color="#fbc5ff", linewidth=2, alpha=0.8, label="RF")
plt.plot(df['Minutes'], df["XGBoost"], marker='', color="#357ffd", linewidth=2, alpha=0.4, label="XGBoost")
plt.plot(df['Minutes'], df["fDNN"], marker='', color="#c197d2", linewidth=4, alpha=1, label="fDNN")

# Add legend
plt.legend(loc=2, ncol=2)
    
# Change xlim
plt.xlim(0,100)
 
# Add titles
plt.title("fDNN vs Random Forest and XGBoost", loc='left', fontsize=16, fontweight=0, color='white')
plt.xlabel("Minutes", color = 'white', size = 14)
plt.ylabel("Accuracy Score", color = 'white', size=14)
plt.xticks(color="white")
plt.yticks(color="white")




### Team Tactics and Automated Subs / Formation Shifts

In [ ]:
from sklearn import preprocessing
from numpy import mean
from numpy import std

In [ ]:
# udfs ----

# function for creating a feature importance dataframe
def imp_df(column_names, importances):
    df = pd.DataFrame({'feature': column_names,
                       'feature_importance': importances}) \
           .sort_values('feature_importance', ascending = False) \
           .reset_index(drop = True)
    return df

# plotting a feature importance dataframe (horizontal barchart)
def var_imp_plot(imp_df, title):
    imp_df.columns = ['feature', 'feature_importance']
    sns.barplot(x = 'feature_importance', y = 'feature', data = imp_df, orient = 'h', color = 'royalblue') \
       .set_title(title, fontsize = 20)

In [ ]:
team_tactics = Player_Aggs_live90.groupby(['matchId', 'teamId']).sum().reset_index()
team_tactics.drop(["playerId", 'weight', 'height', 'xStart', 'yStart', 'homeTeamId', 'awayTeamId', 'gameTime (min)' ], axis = 1, inplace = True)


In [ ]:
def team_result(team_df):
    
    df = pd.merge(team_df, Matches[["homeTeamId", "awayTeamId","matchId", "Result"]], how = "left", on = ["matchId"])
    
    # Classify Match W, L or D for a team
    conditions = [
        ((df['homeTeamId'] == df.teamId) &(df.Result==1)),
        ((df['homeTeamId'] == df.teamId)&(df.Result==0)),
        ((df['homeTeamId'] == df.teamId)&(df.Result==-1)),
        ((df['awayTeamId'] == df.teamId)&(df.Result==1)),
        ((df['awayTeamId'] == df.teamId)&(df.Result==0)),
        ((df['awayTeamId'] == df.teamId)&(df.Result==-1))
        ]

    # create a list of the values we want to assign for each condition
    values = [1,0,-1,-1,0,1]

    # create a new column and use np.select to assign values to it using our lists as arguments
    df['result'] = np.select(conditions, values)
    df.drop(["Result"], axis=1,inplace=True)
    return df

In [ ]:
df = team_result(team_tactics)

df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)

In [ ]:
# df = pd.DataFrame(None, columns = team_tactics.columns)

# for team in teams:
#     test = team_tactics[team_tactics.teamId == team]
#     data = team_result(team, test)
#     df = df.append(data, ignore_index=True)

# df = df.replace([np.inf, -np.inf], np.nan)
# df = df.fillna(0)

In [ ]:
df = df.loc[:, (df != 0).any(axis=0)]

In [ ]:
df.drop(["homeTeamId", "awayTeamId"], inplace=True, axis=1)

In [ ]:
X_all = df.iloc[:,2:-1]
y_all = df.result.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, shuffle=True, test_size= 0.2, train_size=0.8, random_state=3)


In [ ]:
rf = RandomForestClassifier(n_estimators = 500, random_state=123)
rf.fit(X_train, y_train) 

In [ ]:
### Determing player ratings for feature importance

df1 = Player_Aggs_live90.iloc[:,:15]
df2 = Player_Aggs_live90.iloc[:,15:]
df2 = df2.replace([np.inf, -np.inf], np.nan)
df2 = df2.fillna(0)

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(df2.values)
df2 = pd.DataFrame(x_scaled, columns = df2.columns)

base_imp = imp_df(df2.columns, rf.feature_importances_)
xxx = base_imp.transpose()
xxx.columns = xxx.iloc[0]
xxx = xxx.drop(xxx.index[[0]])
xxx = xxx.reindex(sorted(xxx.columns), axis=1)
df2 = df2.reindex(sorted(df2.columns), axis=1)
df2 = df2.multiply(xxx.values)
negatives = df2.filter(regex='Inacc|Lost|Fouls|Fail|Not|Leaving').columns
df2[negatives] = df2[negatives].multiply(-1)
df2["sum"] = df2.iloc[:,:-1].sum(axis=1)
df2 = pd.concat([df1, df2], axis=1)
df2 = pd.merge(df2, df[["matchId", 'teamId', "result"]], on =['matchId', "teamId"], how = 'left' )
#df2 = df2.loc[:, (df2 != 0).any(axis=0)]
df2['playerRatings']=  np.where(df2['gameTime (min)'] ==0, 0, df2['sum'])
df2['playerRatings'] = df2['playerRatings'].multiply(94 / float(df2.playerRatings.nlargest(5)[-1:]))
df2['playerRatings'] = np.where(df2['playerRatings'] > 94.0, 94, df2['playerRatings'])
df2.drop(["sum"], axis=1, inplace=True)
collections.Counter(df2.sort_values("playerRatings", ascending=False).iloc[0:100,:].Position)

In [ ]:
# inspecting player performance in a specific match for automated tactical sub recommendations (underperforming players)
df2[df2.matchId==2499725].sort_values("playerRatings", ascending=False)

In [ ]:
player_values = df2.sort_values("playerRatings", ascending=False)

In [ ]:
test = player_values.groupby(["shortName", 'playerId', "teamId", "Position"]).sum().reset_index().sort_values("playerRatings", ascending=False)



In [ ]:
colors = ['purple','#c197d2','pink'] #navy
fig =plt.figure()
ax = test.head(100).groupby('Position')["playerRatings"].nunique().plot(kind='bar',  color=colors, )
ax.patch.set_facecolor('#241C24')
fig.patch.set_facecolor('#241C24')
ax.tick_params(axis='x', colors='white',size =16 )
ax.tick_params(axis='y', colors='white', size=16)
plt.ylabel("Number of Players", color = "white", size = 12)
plt.xlabel("Positions", color = "white", size=12)
plt.show()


In [ ]:
test["PR_perMatch"] = 100*test["playerRatings"] / test["gameTime (min)"] 

In [ ]:
#test.iloc[2:3,0:1] =  "M.Hamsik"


In [ ]:
test["PR_perMatch"] = test["PR_perMatch"].multiply(94 / float(test.PR_perMatch.max()))# normalize player ratings per match


In [ ]:
try:
    test = test.rename(columns= {"shortName": "Name"})
except:
    pass

try:
    test = test.rename(columns= {"gameTime (min)": "Mins Played"})
except:
    pass

In [ ]:
test.sort_values("playerRatings", ascending=False).round(1)[["Name", "Position", "Mins Played", "playerRatings"]].head(5)

##### Feature Importance

In [ ]:
# inspecting feature importance from random forest
df_feature_importance = pd.DataFrame(rf.feature_importances_, index=df2.iloc[:,15:-2].columns, columns=['feature importance']).sort_values('feature importance', ascending=False)



In [ ]:
(pd.Series(rf.feature_importances_, index=X_train.columns)
   .nlargest(4)
   .plot(kind='barh')) 
#e6b32d

In [ ]:
fig = plt.figure()
fig.patch.set_facecolor('blue')
fig.patch.set_alpha(0.6)

feat_imp_data = sorted(list(zip(X_train.columns, rf.feature_importances_)), key=lambda datum: datum[1], reverse=False)[0:10]

# Unzip the values and labels
widths = [x[1] for x in feat_imp_data]
yticks = [x[0] for x in feat_imp_data]

n_features = int(X_train.shape[1])

ax = fig.add_subplot(111)
ax.patch.set_facecolor('orange')
ax.patch.set_alpha(1.0)

plt.barh(range(n_features),widths, align='center')
plt.yticks(np.arange(n_features), X_train.columns) 
plt.show()

In [ ]:
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(X_all.values)
# df = pd.DataFrame(x_scaled, columns = X_all.columns)


In [ ]:
a, b =X_train.columns, df2.columns
(a | b).difference(a & b)

##### Tactical sub in

In [ ]:
Player_Aggs_live90[Player_Aggs_live90.teamId==1612].matchId.unique()

In [ ]:
##### univariate regression
##### Identify opposition weakness in a match
##### Identify optimal player for tactical sub to replace our weak player and exploit their weakness
##### Identify formation that gives opposition toughest time (label formations below)

In [ ]:
# identify our weakest player
df2[(df2.matchId==2499773) & (df2.teamId==1612) &(df2.Position!="GKP") &(df2.playerRatings!=0)].sort_values("playerRatings", ascending=False).iloc[-1:,:]

# identify opposition weakness in style and tactics


# replace with most similar player or exploitative player



In [ ]:
df2.groupby(['matchId', 'teamId', "result"]).sum().reset_index().head(20)

In [ ]:
# calcualte team performance ratings 
# df2['teamRatings'] = df2['playerRatings'].multiply(df2['gameTime (min)'])
# team_perf = df2.groupby(['matchId', 'teamId']).sum().reset_index()
# team_perf.drop(["Result","playerRatings","playerId", 'weight', 'height', 'xStart', 'yStart', 'homeTeamId', 'awayTeamId', 'gameTime (min)' ], axis = 1, inplace = True)
# pd.merge(team_perf, 

In [ ]:
# sns.boxplot(x="Result", y="teamRatings", data=team_perf);


In [ ]:
df.sample(n=10,axis='columns',replace=True).head()

In [ ]:
##### WEAKNESSES (when they don't win)
X_all = df[(df.teamId ==1612) &(df.result!=1) ]
y_all = df[(df.teamId ==1612) &(df.result!=1)].iloc[:,-1:]

In [ ]:
X_all = X_all.iloc[:,2:-1]


In [ ]:
rf = RandomForestClassifier(n_estimators = 500, random_state=123)
rf.fit(X_all, y_all)

In [ ]:
df_feature_importance = pd.DataFrame(rf.feature_importances_, index=df.iloc[:,2:-1].columns, columns=['feature importance']).sort_values('feature importance', ascending=False)


In [ ]:
fig = plt.figure()
df_feature_importance[0:10].plot(kind='bar');
#fig.savefig('temp.png', transparent=True)
plt.show()

In [ ]:
# Player_Aggs3 = Player_Aggs_live50[Player_Aggs_live50.Position!="GKP"]
# Player_Aggs3 = Player_Aggs3.loc[:, ~Player_Aggs3.columns.isin(["xStart","height","homeAway","yStart", "weight", "name","Result", 'matchId', 'homeTeamId', 'awayTeamId'])]

# Player_Aggs3 = Player_Aggs3.groupby(['playerId' ,'teamId',]).sum().reset_index()

# PV = Player_Aggs3.playerId
# TV = Player_Aggs3.teamId

# Player_Aggs3 = Player_Aggs3.iloc[:,2:].div(Player_Aggs3["gameTime (min)"], axis=0) *90
# Player_Aggs3 = pd.concat([PV, TV, Player_Aggs3], axis =1)

# # Player_Aggs3  = pd.merge(Player_Aggs2[["playerId","teamId"]], Player_Aggs3, how = 'left', on= 'playerId')
# # Player_Aggs3 = Player_Aggs3.drop_duplicates()

# Player_Aggs3.drop(["gameTime (min)"], inplace = True, axis =1)

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram


In [ ]:
# fix diabates last name for visual
test[test.teamId==1610].iloc[-7:-6,0:1] = "F.Diabate"

# test.iloc[3:4,0:1] =  "M.Hamsik"

In [ ]:
df3 = test[(test.teamId==1639) & (test.Position!="GKP")].iloc[:,:-2]
df3 = df3.fillna(0)
# df = Player_Aggs3.loc[:, ~Player_Aggs3.columns.isin(["homeAway","shortName", "name","Result", 'matchId', 'homeTeamId', 'awayTeamId', 'gameTime (min)'])]
# df = df[df.teamId==1609]
df3.drop(['teamId'], axis=1, inplace=True)
df3 = df3.replace([np.inf, -np.inf], np.nan).fillna(mean)

df_y  = pd.merge(df3['playerId'], Players[["shortName", "playerId", "teamId"]], how = 'left', on='playerId')[["shortName"]]
df_y = df_y.fillna("none")
y = df_y.values
X  = df3.iloc[:,11:].values

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, stratify=y)
# knn = KNeighborsClassifier(n_neighbors=5)
# knn.fit(X_train, y_train)
# print(knn.score(X_test, y_test)) # prints 0.87 - i.e. 87% accurate
# walcott = df[df.playerId==7879].drop('playerId', axis=1).values
# salah = df[df.playerId==120353].drop('playerId', axis=1).values
# print(knn.predict(walcott)) # prints ['midfield']
# print(knn.predict(salah))   # prints ['defence']




In [ ]:
Teams[Teams.name.str.contains("Wolves")]

In [ ]:
import scipy.cluster.hierarchy as shc

ax = plt.figure(figsize=(12, 5))  
plt.title("Dendrograms")  
dend = shc.dendrogram(shc.linkage(X, method='ward'),labels=y, leaf_rotation=0, orientation="left")


#ax = df_feature_importance[0:10].plot(kind='bar', color="#F0BE33")


# ax.patch.set_facecolor('#241C24')
# plt.y_ticks( color = "white")
#ax.tick_params(axis='x', colors='white',size =14 )
#.tick_params(axis='y', colors='white', size=14)
#plt.ylabel("Feature Importance", color = "white")

#         fig, ax = plt.subplots(figsize=(15, 7))  # set size
#         ax = dendrogram(linkage_matrix, **kwargs)
#         plt.tick_params(axis='x', bottom='off', top='off', labelbottom='off')
#         plt.tight_layout()
#         plt.show()

In [ ]:
try:
    os.chdir()
except:
    pass

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                    counts]).astype(float)
    print(len(counts))
    # Plot the corresponding dendrogram
    shc.dendrogram(linkage_matrix, **kwargs)


# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)

model = model.fit(X)
plt.title('Hierarchical Clustering Dendrogram')
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode='level', p=8)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

In [ ]:
Player_Aggs3[Player_Aggs3.playerId==7879] # walcott
Player_Aggs3[Player_Aggs3.playerId==120353] # salah

##### Own Team

In [ ]:
teamTest.loc[:, ~teamTest.columns.isin(["name","Result",'teamId', 'matchId', 'homeTeamId', 'awayTeamId', 'gameTime (min)'])]


In [ ]:
Events_England.drop[("subEventId",), ]


tentacion

In [ ]:
# import datetime
# while datetime.datetime.now().hour < 13:
#     x = 1+1

In [ ]:
# fig, ax = plt.subplots()
# fig.set_size_inches(14,4)
# plt.subplot(121)
# sns.kdeplot(Events_England["xStart"], Events_England['yStart'], shade = True)
# plt.show()

In [ ]:
### Successful Vertical Yards Now Merge

# df = Events_England.loc[Events_England.Result == "Success" ].groupby(['matchId','playerId'])['attackMetres'].sum().reset_index()
# df2 = Events_England.loc[Events_England.Result == "Failure" ].groupby(['matchId','playerId'])['attackMetres'].sum().reset_index()

# with names, positions and clubs for reference
#df = pd.merge(Events_England.loc[Events_England.Result == "Success" ].groupby(['matchId','playerId'])['attackYards'].sum().reset_index(), Players[['playerId', 'shortName', "Position", 'clubName']], on = 'playerId').sort_values(by ='attackYards', ascending = False)
#df2 = pd.merge(Events_England.loc[Events_England.Result == "Failure" ].groupby(['matchId','playerId'])['attackYards'].sum().reset_index(), Players[['playerId', 'shortName', "Position", 'clubName']], on = 'playerId').sort_values(by ='attackYards', ascending = False)

# .sort_values(by ='attackYards', ascending = False)



In [ ]:
# dataframes of brute sum of attacking yard attempts without success or failure measures

#df  = pd.merge(Events_England.groupby(['matchId','playerId'])['attackYards'].agg(['sum']).reset_index(), Players[['playerId', 'shortName', "Position", 'clubName']], on ='playerId').sort_values(by ='sum', ascending = False).rename(columns={'sum':'attackYards'})
# df = Events_England.groupby(['matchId','playerId'])['attackYards'].agg(['sum']).reset_index().sort_values(by ='sum', ascending = False).rename(columns={'sum':'attackYards'})


In [ ]:
Player_Aggs = pd.merge(Player_Aggs, Matches[['matchId', "Result"]], how = 'left', on = 'matchId' )

#### Live match testing

##### Team Aggregates and Clustering Tactics

In [ ]:
dropcols = ['playerId',
'weight','height','xStart','yStart','homeTeamId','awayTeamId','gameTime (min)','FKCrossAccuracy','penaltiesConversion',
'low_left_save_efficiency','mid_left_save_efficiency','high_left_save_efficiency','low_center_save_efficiency',
'mid_center_save_efficiency','high_center_save_efficiency','low_right_save_efficiency','mid_right_save_efficiency',
'high_right_save_efficiency','shotAccuracy','shortPassAccuracy','longPassAccuracy','shortLongPassRatio',
'Result']

dropcols_live = dropcols[0:9]


In [ ]:
Team_Aggs_Matches = Player_Aggs.groupby(['matchId', 'teamId' ]).sum().reset_index()
Team_Aggs_Matches.drop(columns = dropcols, inplace=True, axis=1)
Team_Aggs_Matches = pd.merge(Team_Aggs_Matches, Matches[['matchId', "Country"]], on='matchId', how = 'left')
Teams['Country'] = [row['name'] for row in Teams.area]
Team_Aggs = Team_Aggs_Matches.groupby(['teamId']).sum().reset_index()
Team_Aggs.drop(columns = 'matchId', inplace=True, axis=1)
Team_Aggs = pd.merge(Team_Aggs, Teams[['teamId', 'Country']], how = 'left', on = 'teamId')

In [ ]:
Team_Aggs_Matches_live = Player_Aggs_live.groupby(['matchId', 'teamId' ]).sum().reset_index()
Team_Aggs_Matches_live.drop(columns = dropcols_live, inplace=True, axis=1)
Team_Aggs_Matches_live = pd.merge(Team_Aggs_Matches_live, Matches[['matchId', "Country"]], on='matchId', how = 'left')
Teams['Country'] = [row['name'] for row in Teams.area]
Team_Aggs_live = Team_Aggs_Matches_live.groupby(['teamId']).sum().reset_index()
Team_Aggs_live.drop(columns = 'matchId', inplace=True, axis=1)
Team_Aggs_live = pd.merge(Team_Aggs_live, Teams[['teamId', 'Country']], how = 'left', on = 'teamId')

In [ ]:
result = Matches_df(Team_Aggs_Matches, Team_Aggs)
result_live = Matches_df(Team_Aggs_Matches_live, Team_Aggs_live)
y_all = result["Result"].values

In [ ]:
X_all_live = result_live.loc[:, ~result_live.columns.isin(["awayCounterAttackGoal", 
                                                           "Result",'teamId', 'matchId', 'homeTeamId', 
                                                           'awayTeamId', 'homeGoal_Value','awayGoal_Value', 
                                                           'awayCornerAssists', 'awayPenaltyNotScored', 'homeCornerAssists'])]
    

In [ ]:
X_all = result.loc[:, ~result.columns.isin(['teamId', 'matchId', 'homeTeamId', 'awayTeamId', 'awayGoal_Value','homegoalsAllowed', 'awaygoalsAllowed'
                                           ,'homeGoal_low_left', 'homeGoal_mid_left', 'awayGoal_mid_right',
                                           'homeGoal_Value', 'homeCrossAssists', 'homeCornerAssists',
                                           'homeGoal_low_center', 'homeTotalAssists','homeRightFootGoal', 'awayRightFootGoal'
                                           ,'homeRightFootGoal', 'awayRightFootGoal',
       'homeLeftFootGoal', 'awayLeftFootGoal', 'homeTotalAssists',
       'awayTotalAssists',"Result", 'awayGoal_low_left', 'awayGoal_low_center',
       'homeGoal_low_right', 'awayGoal_low_right','homeHeader/Body_Goal', 'awayHeader/Body_Goal','awayCrossAssists',
                                           'awayCrossAssists','awayInterceptionGoal',
 'homeInterceptionGoal','awayOwnGoals','homeFKCrossAssists', 'homeGoal_high_left','awayFKCrossAssists',
 'homeOwnGoals','homeGoal_high_right','awayGoal_high_center',
 'awayGoal_high_left','awayInterceptionGoal','homeInterceptionGoal','awayOwnGoals','awayPenaltyNotScored',
 'homeFKCrossAssists','homeGoal_high_left','awayFKCrossAssists',
'awayGoal_mid_left', 'homeGoal_mid_right','homeGoal_mid_center','awayGoal_high_right','awayGoal_mid_center',
 'homeGoal_high_center','awayCornerAssists','homeOwnGoals', 'homeGoal_high_right','awayGoal_high_center'                                      
 'awayGoal_high_left', 'homeCounterAttackGoal', 'awayCounterAttackGoal'])]


In [ ]:
X_all_live = X_all_live.replace([np.inf, -np.inf], np.nan)
X_all_live = X_all_live.fillna(X_all_live.mean())
X_train_live, X_test_live, y_train_live, y_test_live = train_test_split(X_all_live, y_all, shuffle=True, test_size= 0.2, train_size=0.8, random_state=3)


### Experiment: Extracting features of team playing style using Random Forests

In [ ]:
# param_grid = {
#     'n_estimators': [200,400,500, 700,1000,2000]
# }

#### Subset AKA live match trials

In [ ]:
X_all = X_all.replace([np.inf, -np.inf], np.nan)
X_all = X_all.fillna(X_all.mean())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, shuffle=True, test_size= 0.2, train_size=0.8, random_state=3)


In [ ]:
clf = RandomForestClassifier(n_estimators = 1000, random_state=3)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
# CV_rfc = GridSearchCV(estimator=clf, param_grid=param_grid, cv= 10)

In [ ]:
# CV_rfc.fit(X_train, y_train)

In [ ]:
# CV_rfc.best_estimator_

In [ ]:
labels = [-1,0,1]
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, labels = labels))
print('accuracy score: {0:.4f}'.format(accuracy_score(y_test,y_pred)))

In [ ]:
idx = list(clf.feature_importances_.argsort()[0:25])
col = list(X_train.columns[idx])
# modelname.feature_importance_
y = clf.feature_importances_[0:25]
#plot
fig, ax = plt.subplots() 
width = 0.4 # the width of the bars 
ind = np.arange(len(y)) # the x locations for the groups
ax.barh(ind, y, width, color='green')
ax.set_yticks(ind+width/10)
ax.set_yticklabels(col, minor=False)
plt.title('Feature importance in RandomForest Classifier')
plt.xlabel('Relative importance')
plt.ylabel('feature') 
plt.figure(figsize=(5,5))
fig.set_size_inches(6.5, 4.5, forward=True)

In [ ]:
print(pd.crosstab(y_test, y_pred, rownames=['Actual Result'], colnames=['Predicted Result']))
class_name = list(set(y_test))
print(class_name)

In [ ]:


# estimator_nonlimited = clf.estimators_[5]
# fn = list(X_train.columns)
# # from sklearn.tree import export_graphviz
# # export_graphviz(estimator_limited, out_file='tree_limited.dot', feature_names = iris.feature_names,
# #                 class_names = iris.target_names,
# #                 rounded = True, proportion = False, precision = 2, filled = True)

# export_graphviz(estimator_nonlimited, out_file='tree_nonlimited.dot', feature_names = fn,
#                 class_names = ["W", "L", 'D'],
#                 rounded = True, proportion = False, precision = 2, filled = True)


# import pydot

# (graph,) = pydot.graph_from_dot_file('tree_nonlimited.dot')
# graph.write_png('tree_nonlimited.png')
# graph.draw('tree_nonlimited.png')


# # !dot -Tpng tree_limited.dot -o tree_nonlimited.png -Gdpi=600
# # from IPython.display import Image
# # Image(filename = 'tree_nonlimited.png')



In [ ]:
from dtreeviz.trees import *
import dtreeviz
from IPython.core.display import display, HTML
from dtreeviz.trees import dtreeviz

fn = list(X_train.columns)
cn = [-1,0,1]

dtree = tree.DecisionTreeClassifier(random_state=0)
dtree = dtree.fit(X_train, y_train)
viz = dtreeviz(dtree,
               X_train,
               y_train,
               feature_names=fn, 
               class_names=cn,
               fancy=False)

viz.view()

# display(HTML(viz.svg()))

In [ ]:
y_predict_proba = clf.predict_proba(X_test)

# Compute ROC curve and ROC AUC for each class
n_classes = 3
fpr = dict()
tpr = dict()
roc_auc = dict()
all_y_test_i = np.array([])
all_y_predict_proba = np.array([])
for i in [-1,0,1]:
    y_test_i = list(map(lambda x: 1 if x == i else 0, list(y_test)))
    if len(all_y_test_i) >0:
        all_y_test_i = np.concatenate([all_y_test_i, y_test_i])
    else:
        all_y_test_i = y_test_i
    all_y_predict_proba = np.concatenate([all_y_predict_proba, y_predict_proba[:, i]])
    fpr[i], tpr[i], _ = roc_curve(y_test_i, y_predict_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["average"], tpr["average"], _ = roc_curve(all_y_test_i, all_y_predict_proba)
roc_auc["average"] = auc(fpr["average"], tpr["average"])


# Plot average ROC Curve
plt.figure()
plt.plot(fpr["average"], tpr["average"],
         label='Average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["average"]),
         color='deeppink', linestyle=':', linewidth=4)

# Plot each individual ROC curve
for i in [-1,0,1]:
    plt.plot(fpr[i], tpr[i], lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# generate a no skill prediction (majority class)
# ns_probs = [0 for _ in range(len(y_test))]
# ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
# plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')

# #Now calculate the AUC for each class separately

y_predict_proba = clf.predict_proba(X_test)
fpr = dict()
tpr = dict()
roc_auc = dict()
all_y_test_i = np.array([])
all_y_predict_proba = np.array([])

WDL = ["Draw","Home Win", "Home Loss"]

for pp in [-1,0,1]:
    fpr, tpr, thresholds = metrics.roc_curve(y_test,  
                     clf.predict_proba(X_test)[:,pp], pos_label = pp)
    plt.plot(fpr, tpr,  marker='.', label= WDL[pp])
    auroc = round(metrics.auc(fpr, tpr),2)
    print('RF',WDL[pp],'--AUC--->',auroc)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()



In [ ]:
n_matches = Matches.shape[0]
n_features = Matches.shape[1] -1
n_homewins =  len(Matches[Matches.Result==1])
win_rate = (float(n_homewins) / (n_matches)) * 100
n_homeloss =  len(Matches[Matches.Result==-1])
loss_rate = (float(n_homeloss) / (n_matches)) * 100
print("Total number of matches: {}".format(n_matches))
print("Number of features: {}".format(n_features))
print('Number of matches won by home side: {}'.format(n_homewins))
print('Win rate of home team {:2f}%'.format(win_rate))
print('Loss rate of home team {:2f}%'.format(loss_rate))
print(collections.Counter(Matches.Result))

In [ ]:
dropcols = [
 'playerId',
 'weight',
 'height',
 'xStart',
 'yStart',
 'homeTeamId',
 'awayTeamId',
 'gameTime (min)',
#  'AccurateAttackMetres',
#  'InaccurateAttackMetres',
#  'Goal_Value',
#  'SimulationFouls',
#  'FoulsCommited',
#  'Clearances',
#  'AccLaunchMetres',
#  'InaccLaunchMetres',
#  'FKKeyCross',
#  'FKCrossAssists',
 'FKCrossAccuracy',
 'penaltiesConversion',
#  'dangerousOpponentHalfRecoveries',
#  'dangerousOwnHalfBallLost',
#  'dangerousDefDuelsLost',
 'low_left_save_efficiency',
 'mid_left_save_efficiency',
 'high_left_save_efficiency',
 'low_center_save_efficiency',
 'mid_center_save_efficiency',
 'high_center_save_efficiency',
 'low_right_save_efficiency',
 'mid_right_save_efficiency',
 'high_right_save_efficiency',
#  'goalsAllowed',
#  'GkSaves',
#  'AccurateHandPass',
#  'InaccurateHandPass',
#  'GKLeavingLineInstance',
#  'SuccessfulAccels',
#  'FailedAccels',
#  'CumAccelerationDist',
#  'AccelsDistIntoFinal3rd',
#  'AccelsDistWithinFinal3rd',
#  'CrossAssists',
#  'RightFootCross',
#  'LeftFootCross',
#  'CrossKeyPass',
#  'AccCrosses',
#  'InaccCrosses',
#  'AccCrossesIntoBox',
#  'InaccCrossesIntoBox',
#  'SlidingTacklesWon',
#  'SlidingTacklesLost',
#  'GroundDefDuelsWon',
#  'GroundDefDuelsLost',
#  'DefensiveAnticipations',
#  'GroundLooseBallsWon',
#  'GroundLooseBallsLost',
#  'GroundAttackDuelsWon',
#  'GroundAttackDuelsLost',
#  'OffensiveTouchesBox',
#  'Goal_low_left',
#  'Goal_mid_left',
#  'Goal_high_left',
#  'Goal_low_center',
#  'Goal_mid_center',
#  'Goal_high_center',
#  'Goal_low_right',
#  'Goal_mid_right',
#  'Goal_high_right',
 'shotAccuracy',
#  'ShotsOnTarget',
#  'ShotsOffTarget',
#  'ShotOpportunities',
#  'FreeKickShotAcc',
#  'FreeKickShotInacc',
#  'PenaltyNotScored',
#  'Header/Body_Goal',
#  'RightFootGoal',
#  'LeftFootGoal',
#  'CounterAttackGoal',
#  'InterceptionGoal',
#  'OwnGoals',
#  'AerialDuelsWon',
#  'AerialDuelsLost',
 'shortPassAccuracy',
 'longPassAccuracy',
 'shortLongPassRatio',
#  'InaccPassesWithinFinalThird',
#  'AccPassesWithinFinalThird',
#  'AccPassesIntoFinalThird',
#  'InaccPassesIntoFinalThird',
#  'AccThroughsIntoFinalThird',
#  'InaccurateSmartPasses',
#  'AccurateSmartPasses',
#  'InaccurateThroughPasses',
#  'AccurateThroughPasses',
#  'AccurateKeyPasses',
#  'InaccurateKeyPasses',
#  'TotalAssists',
#  'CounterAttackInvolvements',
#  'PassLostDangerous',
#  'CounterAttackIntercept',
#  'SuccessfulInterceptions',
#  'InaccurateForwardPasses',
#  'AccurateForwardPasses',
#  'CornerAssists',
#  'CornerOpportunitiesCreated',
#  'SuccessfulCorners',
#  'FailedCorners',
 'Result']

##### KMeans for detecting and visualizing team formations

In [ ]:
# live adaptable formation
def team_formation(event_df, teamId, matchId):
    event_df = event_df[(event_df.teamId==teamId)&(event_df.matchId==matchId)]
    maxtime = event_df.eventSec.max()
    if maxtime > 1200:
        mintime = maxtime - 1200 # last 20mins of match data for formation revelation
    else:
        mintime = 0
    x1 = np.array(event_df[(event_df.eventSec > mintime)].xStart) 
    x2 = np.array(event_df[(event_df.eventSec > mintime)].yStart)
    X = np.array(list(zip(x1, x2))).reshape(len(x1), 2) 
    return x1, x2, X

In [ ]:
#Creating the data 
x1, x2, X = team_formation(Events_England, 1609, 2499719)


In [ ]:
kmeans = KMeans(n_clusters=11)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=11, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)

In [ ]:
fig = plt.figure()
fig.set_size_inches(10, 7)
ax=fig.add_subplot(1,1,1)

# pitch borderline and centrer line
plt.plot([0,0],[0,100], color="white")
plt.plot([0,100],[100,100], color="white")
plt.plot([100,100],[100,0], color="white")
plt.plot([100,0],[0,0], color="white")
plt.plot([50,50],[0,100], color="white") # center line

# left penalty area
plt.plot([16.5,16.5],[75,25],color="white")
plt.plot([0,16.5],[75,75],color="white")
plt.plot([16.5,0],[25,25],color="white")

#Right Penalty Area
plt.plot([83.5,100],[75,75],color="white")
plt.plot([83.5,83.5],[75,25],color="white")
plt.plot([83.5,100],[25,25],color="white")

#Left 6-yard Box
plt.plot([0,5.5],[64,64],color="white")
plt.plot([5.5,5.5],[64,36],color="white")
plt.plot([5.5,0.5],[36,36],color="white")

#Right 6-yard Box
plt.plot([100,94.5],[64,64],color="white")
plt.plot([94.5,94.5],[64,36],color="white")
plt.plot([94.5,100],[36,36],color="white")

#Prepare Circles
centreCircle = plt.Circle((50,50),9.15,color="white",fill=False)
centreSpot = plt.Circle((50,50),0.6,color="white")
leftPenSpot = plt.Circle((11,50),0.6,color="white")
rightPenSpot = plt.Circle((89,50),0.6,color="white")

# zones
plt.plot([100,0],[25,25],color="white", linestyle='--') 
plt.plot([100,0],[75,75],color="white", linestyle='--') 
plt.plot([100,0],[25,25],color="white", linestyle='--') 
plt.plot([33,33],[75,25],color="white", linestyle='--')
plt.plot([67,67],[75,25],color="white", linestyle='--')
plt.plot([33,33],[100,75],color="white", linestyle='--')
plt.plot([16.5,16.5],[100,75],color="white", linestyle='--')
plt.plot([67,67],[100,75],color="white", linestyle='--')
plt.plot([83.5,83.5],[100,75],color="white", linestyle='--')
plt.plot([33,33],[0,25],color="white", linestyle='--')
plt.plot([16.5,16.5],[0,25],color="white", linestyle='--')
plt.plot([67,67],[0,25],color="white", linestyle='--')
plt.plot([83.5,83.5],[0,25],color="white", linestyle='--')
plt.plot([16.5,83.5],[50,50],color="white", linestyle='--')
plt.plot([33,33],[0,25],color="white", linestyle='--')
plt.plot([0,16.5],[64,64],color="white", linestyle='--')
plt.plot([0,16.5],[36,36],color="white", linestyle='--')
plt.plot([83.5,100],[64,64],color="white", linestyle='--')
plt.plot([83.5,100],[36,36],color="white", linestyle='--')

# Fill with green
green = patches.Rectangle((0, 0), 100, 100, linewidth=0.1,
                             edgecolor='r', facecolor='darkgreen', zorder=0, alpha = 0.8)
ax.add_patch(green)

#Draw Circles
ax.add_patch(centreCircle)
ax.add_patch(centreSpot)
ax.add_patch(leftPenSpot)
ax.add_patch(rightPenSpot)

#Prepare Arcs
leftArc = Arc((11,50),height=18.3,width=18.3,angle=0,theta1=310,theta2=50,color="white")
rightArc = Arc((89,50),height=18.3,width=18.3,angle=0,theta1=130,theta2=230,color="white")

#Draw Arcs
ax.add_patch(leftArc)
ax.add_patch(rightArc)

# #Tidy Axes
# plt.axis('off')

# #K-Means Live formation
# plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=30, cmap='viridis')
# plt.scatter(centers[:, 0], centers[:, 1], c='red', s=600, alpha=0.5)

#Heat Map of Player and Action Type
team = Events_Spain[(Events_Spain.playerId == 3359) & (Events_Spain.matchId == 2565907)  & (Events_Spain.subEventName=="Shot") ]
sns.kdeplot(team["xStart"],team["yStart"], shade=True)
# team2 = Events_Spain[(Events_Spain.playerId == 3322) & (Events_Spain.matchId == 2565907)  & (Events_Spain.subEventName=="Shot") ]
# sns.kdeplot(team2["xStart"],team2["yStart"], shade=True, color="Red")

# plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=11, cmap='viridis')
# centers = kmeans.cluster_centers_
# plt.scatter(centers[:, 0], centers[:, 1], c='red', s=400, alpha=0.5)

plt.ylim(100, 0)
plt.xlim(0, 100)





In [ ]:
#Creating the data 
x1 = np.array(Player_Aggs[Player_Aggs['gameTime (min)']>45].xStart) 
x2 = np.array(Player_Aggs[Player_Aggs['gameTime (min)']>45].yStart)
X = np.array(list(zip(x1, x2))).reshape(len(x1), 2) 

In [ ]:
distortions = [] 
inertias = [] 
mapping1 = {} 
mapping2 = {} 
K = range(1,15) 
  
for k in K: 
    #Building and fitting the model 
    kmeanModel = KMeans(n_clusters=k).fit(X) 
    kmeanModel.fit(X)     
      
    distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_, 
                      'euclidean'),axis=1)) / X.shape[0]) 
    inertias.append(kmeanModel.inertia_) 
  
    mapping1[k] = sum(np.min(cdist(X, kmeanModel.cluster_centers_, 
                 'euclidean'),axis=1)) / X.shape[0] 
    mapping2[k] = kmeanModel.inertia_

In [ ]:
plt.plot(K, inertias, 'bx-') 
plt.xlabel('Values of K') 
plt.ylabel('Inertia') 
plt.title('The Elbow Method using Inertia') 
plt.show() 

In [ ]:
plt.plot(K, distortions, 'bx-') 
plt.xlabel('Values of K') 
plt.ylabel('Distortion') 
plt.title('The Elbow Method using Distortion') 
plt.show() 

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

In [ ]:
# This is what four pitch area location clusters look like

In [ ]:

plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)

In [ ]:
#### Traditional 11 pitch positions

In [ ]:
# Liverpool formation last twenty minutes versus Man Utd at Anfield
np.array(Events_England[(Events_England.teamId == 1612) (Events_England.matchId==2499793) & (Events_England.eventSec <3700) * (Events_England.eventSec > 2400)].xStart)

In [ ]:
# Creating the data for Liverpool live formation at 60mins

x1 = np.array(Events_England[(Events_England.teamId == 1612) &(Events_England.matchId==2499793) & (Events_England.eventSec <3700) & (Events_England.eventSec > 1900)].xStart)
x2 = np.array(Events_England[(Events_England.teamId == 1612) &(Events_England.matchId==2499793) & (Events_England.eventSec <3700) & (Events_England.eventSec > 1900)].yStart)
X = np.array(list(zip(x1, x2))).reshape(len(x1), 2) 

In [ ]:
kmeans = KMeans(n_clusters=11)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)


In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=800, alpha=0.5)

In [ ]:
kmeans = KMeans(n_clusters=22)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)

### Comparing Two Teams in a head to head

In [ ]:
Teams[Teams["name"].str.contains('Barcelona')]

In [ ]:
#Creating the data
manUtdBad = Player_Aggs[(Player_Aggs['gameTime (min)']>45) & (Player_Aggs.teamId ==1611) &(((Player_Aggs.homeAway=="home") & (Player_Aggs.Result==-1)) | ((Player_Aggs.homeAway=="away") & (Player_Aggs.Result==1)))]
manUtdGood = Player_Aggs[(Player_Aggs['gameTime (min)']>45) & (Player_Aggs.teamId ==1611) &(((Player_Aggs.homeAway=="home") & (Player_Aggs.Result==1)) | ((Player_Aggs.homeAway=="away") & (Player_Aggs.Result==-1)))]
liverpoolGood = Player_Aggs[(Player_Aggs['gameTime (min)']>45) & (Player_Aggs.teamId ==1612) &(((Player_Aggs.homeAway=="home") & (Player_Aggs.Result==1)) | ((Player_Aggs.homeAway=="away") & (Player_Aggs.Result==-1)))]
liverpoolBad = Player_Aggs[(Player_Aggs['gameTime (min)']>45) & (Player_Aggs.teamId ==1612) &(((Player_Aggs.homeAway=="home") & (Player_Aggs.Result==-1)) | ((Player_Aggs.homeAway=="away") & (Player_Aggs.Result==1)))]
barcelonaBad = Player_Aggs[(Player_Aggs['gameTime (min)']>45) & (Player_Aggs.teamId ==676) &(((Player_Aggs.homeAway=="home") & (Player_Aggs.Result==-1)) | ((Player_Aggs.homeAway=="away") & (Player_Aggs.Result==1)))]
barcelonaGood = Player_Aggs[(Player_Aggs['gameTime (min)']>45) & (Player_Aggs.teamId ==676) &(((Player_Aggs.homeAway=="home") & (Player_Aggs.Result==1)) | ((Player_Aggs.homeAway=="away") & (Player_Aggs.Result==-1)))]


x1 = np.array(manUtdBad.xStart) 
x2 = np.array(manUtdBad.yStart)
X = np.array(list(zip(x1, x2))).reshape(len(x1), 2) 

In [ ]:
kmeans = KMeans(n_clusters=11)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')
centers2 = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)

In [ ]:
x1 = np.array(manUtdGood.xStart) 
x2 = np.array(manUtdGood.yStart)
X = np.array(list(zip(x1, x2))).reshape(len(x1), 2) 


In [ ]:
kmeans = KMeans(n_clusters=11)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)
# plt.scatter(centers2[:, 0], centers2[:, 1], c='red', s=200, alpha=0.5)


In [ ]:
manUtd = Player_Aggs[(Player_Aggs['gameTime (min)']>45) & (Player_Aggs.teamId ==1611)]
barca = Player_Aggs[(Player_Aggs['gameTime (min)']>45) & (Player_Aggs.teamId ==676)]       

In [ ]:
## Man Utd
x1 = np.array(manUtd.xStart) 
x2 = np.array(manUtd.yStart)
X = np.array(list(zip(x1, x2))).reshape(len(x1), 2) 

kmeans = KMeans(n_clusters=11)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)

In [ ]:
## Barcelona
x1 = np.array(barca.xStart) 
x2 = np.array(barca.yStart)
X = np.array(list(zip(x1, x2))).reshape(len(x1), 2) 

kmeans = KMeans(n_clusters=11)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)

In [ ]:
# Events_England = pd.read_csv('eventsEvents_England_processed')

In [ ]:
# Player_Aggs_live = pd.read_csv('eventsPlayer_Aggs_live.csv')
# Player_Aggs = pd.read_csv('eventsPlayer_Aggs.csv')

In [ ]:
# Player_Aggs_live.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
# def Matches_df_fDNN(TAM, TA):
    
#     df2 = TAM.groupby('matchId').agg(list).reset_index()
#     df2.drop(["homeTeamId", "awayTeamId"], inplace=True, axis=1)
#     df3 = pd.DataFrame(None, columns = list(TA.columns))

#     for index, col in enumerate(list(TAM.columns)):
#         df3[col] = df2.iloc[:,index]

#     df4 = pd.concat([df3, pd.DataFrame(df3['teamId'].to_list(), columns=['team1','team2'])], axis=1, sort = False)
#     result = pd.merge(df4, Matches[['matchId', "homeTeamId", "awayTeamId"]], on='matchId', how='inner')
#     #df4.drop(["Country"], axis=1, inplace=True)

#     for var in list(result.columns)[2:-5]:
#         result = pd.concat([result, pd.DataFrame(df3[var].to_list(), columns=["1_" + var, "2_" + var])], axis=1, sort = False)

#         # Correctly apply allocate home and away aggregate statistics to correct teams
#         result['home'+var] = (
#             np.select(
#                 condlist=[result['team1'] == result['homeTeamId'], result['team2'] == result['homeTeamId']], 
#                 choicelist=[result["1_" + var], result["2_" + var]]))

#         result['away'+var] = (
#             np.select(
#                 condlist=[result['team1'] == result['awayTeamId'], result['team2'] == result['awayTeamId']], 
#                 choicelist=[result["1_" + var], result["2_" + var]]))

#         # # drop useless columns that have been replaced
#         result.drop([ var, "1_" + var, "2_" + var], inplace = True, axis = 1 )

#     # # drop useless columns that have been replaced
#     result.drop(['teamId',"team1", "team2"], inplace = True, axis = 1 )
#     result = pd.merge(result, Matches[['matchId', "Result"]], on='matchId', how = 'left')
#     return result

In [ ]:
# def Matches_df(TAM, TA):
    
#     df2 = TAM.groupby('matchId').agg(list).reset_index()
#     df2.drop(["homeTeamId", "awayTeamId"], inplace=True, axis=1)
#     df3 = pd.DataFrame(None, columns = list(TA.columns))

#     for index, col in enumerate(list(TAM.columns)):
#         df3[col] = df2.iloc[:,index]

#     df4 = pd.concat([df3, pd.DataFrame(df3['teamId'].to_list(), columns=['team1','team2'])], axis=1, sort = False)
#     result = pd.merge(df4, Matches[['matchId', "homeTeamId", "awayTeamId"]], on='matchId', how='inner')
#     #df4.drop(["Country"], axis=1, inplace=True)

#     for var in list(result.columns)[1:-6]:
#         result = pd.concat([result, pd.DataFrame(df3[var].to_list(), columns=[var+"1",var+"2"])], axis=1, sort = False)

#         # Correctly apply allocate home and away aggregate statistics to correct teams
#         result['home'+var] = (
#             np.select(
#                 condlist=[result['team1'] == result['homeTeamId'], result['team2'] == result['homeTeamId']], 
#                 choicelist=[result[var+"1"], result[var+"2"]]))

#         result['away'+var] = (
#             np.select(
#                 condlist=[result['team1'] == result['awayTeamId'], result['team2'] == result['awayTeamId']], 
#                 choicelist=[result[var+"1"], result[var+"2"]]))

#         # # drop useless columns that have been replaced
#         result.drop([ var, var+"1", var+"2"], inplace = True, axis = 1 )

#     # # drop useless columns that have been replaced
#     result.drop(['teamId',"team1", "team2"], inplace = True, axis = 1 )
#     result = pd.merge(result, Matches[['matchId', "Result"]], on='matchId', how = 'left')
#     return result

In [ ]:
# #### ORIGINAL

# dropcols = ['playerId',
# 'weight','height','xStart','yStart','homeTeamId','awayTeamId','gameTime (min)','penaltiesConversion',
# 'low_left_save_efficiency','mid_left_save_efficiency','high_left_save_efficiency','low_center_save_efficiency',
# 'mid_center_save_efficiency','high_center_save_efficiency','low_right_save_efficiency','mid_right_save_efficiency',
# 'high_right_save_efficiency','shotAccuracy','shortPassAccuracy','longPassAccuracy','shortLongPassRatio',
# 'Result'][0:7]

# dropcols_live =  dropcols[0:7]

# Team_Aggs_Matches = Player_Aggs2.groupby(['matchId', 'teamId' ]).sum().reset_index()
# Team_Aggs_Matches.drop(columns = dropcols, inplace=True, axis=1)

# Team_Aggs = Team_Aggs_Matches.groupby(['teamId']).sum().reset_index()
# Team_Aggs.drop(columns = ['matchId'], inplace=True, axis=1)


In [ ]:
# We performed this and found our best params which are used in the model above

In [ ]:
##### RANDOM FORESTS WITh GRIDSEARCH

from sklearn.model_selection import StratifiedKFold

In [ ]:
# # summarize results

# print("best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
idx = list(clf.feature_importances_.argsort()[0:25])
col = list(X_train.columns[idx])

# modelname.feature_importance_

y = clf.feature_importances_[0:25]

# plot
fig, ax = plt.subplots() 
width = 0.4 # the width of the bars 
ind = np.arange(len(y)) # the x locations for the groups
ax.barh(ind, y, width, color='green')
ax.set_yticks(ind+width/10)
ax.set_yticklabels(col, minor=False)
plt.title('Feature importance in RandomForest Classifier')
plt.xlabel('Relative importance')
plt.ylabel('feature') 
plt.figure(figsize=(5,5))
fig.set_size_inches(6.5, 4.5, forward=True)

In [ ]:
df1 = Player_Aggs_live30.iloc[:,:15]
df2 = Player_Aggs_live30.iloc[:,15:]
df2 = df2.replace([np.inf, -np.inf], np.nan)
df2 = df2.fillna(0)

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(df2.values)
df2 = pd.DataFrame(x_scaled, columns = df2.columns)

base_imp = imp_df(df2.columns, rf.feature_importances_)
xxx = base_imp.transpose()
xxx.columns = xxx.iloc[0]
xxx = xxx.drop(xxx.index[[0]])
xxx = xxx.reindex(sorted(xxx.columns), axis=1)
df2 = df2.reindex(sorted(df2.columns), axis=1)
df2 = df2.multiply(xxx.values)
df2[negatives] = df2[negatives].multiply(-1)
df2["sum"] = df2.sum(axis=1)
df2 = pd.concat([df1, df2], axis=1)
df2 = pd.merge(df2, df[["matchId", 'teamId', "Result"]], on =['matchId', "teamId"], how = 'left' )
#df2 = df2.loc[:, (df2 != 0).any(axis=0)]
df2['playerRatings']=  np.where(df2['gameTime (min)'] ==0, 0, df2['sum'])
df2['playerRatings'] = df2['playerRatings'].multiply(94 / df2.playerRatings.max())
df2.drop(["sum"], axis=1, inplace=True)
collections.Counter(df2.sort_values("playerRatings", ascending=False).iloc[0:100,:].Position)

In [ ]:
from dtreeviz.trees import *
import dtreeviz
from IPython.core.display import display, HTML
from dtreeviz.trees import dtreeviz

fn = list(X_all.columns)
cn = [-1,0,1]

_ = tree.plot_tree(clf.estimators_[0], feature_names=X_all.columns, filled=True, max_depth=10)
